In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!python --version

!pip install torch==2.3.1+cu121 torchvision --extra-index-url https://download.pytorch.org/whl/cu121


In [ ]:

!pip install -q  pycocotools tqdm torchmetrics lxml scipy
!pip install -U "datasets>=2.17.0"  "pyarrow>=14.0"

In [ ]:
!pip install -U cmake ninja wheel
!pip install --no-binary=mmcv mmcv==2.2.0

In [ ]:
!pip uninstall -y numpy

!pip install numpy==1.26.4

In [ ]:
%%bash

pip install -U cmake ninja wheel

git clone --depth 1 --branch v0.14.6 https://github.com/SHI-Labs/NATTEN.git
cd NATTEN

export FORCE_CUDA=1
export TORCH_CUDA_ARCH_LIST="8.0"

pip install .

cd ..

In [ ]:
!rm -rf DAT && git clone -q https://github.com/LeapLabTHU/DAT.git

In [ ]:
%%bash
git clone --depth 1 https://github.com/OpenGVLab/DCNv4.git
cd DCNv4/DCNv4_op
export FORCE_CUDA=1
export TORCH_CUDA_ARCH_LIST="8.0"
python -m pip install . --no-build-isolation -v
cd ../..


In [ ]:

# Import from actual DAT and DCNv4 modules
import importlib
from DAT.models.dat import DAT, LayerScale, TransformerStage
from DAT.models.dat_blocks import LayerNormProxy, TransformerMLP, TransformerMLPWithConv
from DAT.models.dat_blocks import LocalAttention, DAttentionBaseline, ShiftWindowAttention, PyramidAttention
from DCNv4.modules.dcnv4 import DCNv4
dat_mod = importlib.import_module("DAT.models.dat")
from torch.utils.checkpoint import checkpoint
from __future__ import annotations
import torch
import torch.nn as nn
from timm.models.layers import DropPath
import torch.nn.functional as F
import math
import numpy as np
from mmcv.ops.multi_scale_deform_attn import MultiScaleDeformableAttention
from timm.models.layers import create_act_layer
from typing import Optional, List, Dict, Tuple,  Union



class NormFactory:
    SUPPORTED = {"bn", "gn", "lnp"}

    def __init__(self, kind: str = "gn", gn_groups: int = 32):
        kind = kind.lower()
        if kind not in self.SUPPORTED:
            raise ValueError(f"kind must be one of {self.SUPPORTED}")
        self.kind = kind
        self.gn_groups = gn_groups

    def __call__(self, num_feat: int) -> nn.Module:
        if self.kind == "bn":
            return nn.BatchNorm2d(num_feat)
        if self.kind == "gn":
            g = math.gcd(self.gn_groups, num_feat) or 1
            return nn.GroupNorm(g, num_feat)

        if self.kind == "lnp":
            return LayerNormProxy(num_feat)




# -----------------------------------------------------------------------------
# 1. CrossScaleInjection
# -----------------------------------------------------------------------------

class CrossScaleInjection(nn.Module):
    """Cross‑scale feature enjeksiyonu – kanal‑başına gating versiyonu"""
    def __init__(self, low_ch: int, high_ch: int):
        super().__init__()
        self.align = nn.Conv2d(low_ch, high_ch, 1, bias=False)
        self.norm = LayerNormProxy(high_ch)

        # Channel based weight; starting 0.1
        self.weight = nn.Parameter(torch.ones(1, high_ch, 1, 1) * 0.15)

        nn.init.kaiming_normal_(self.align.weight, mode="fan_out", nonlinearity="relu")

    def forward(self, low_res: torch.Tensor, high_res: torch.Tensor) -> torch.Tensor:
        low_aligned = self.align(low_res)
        low_up = F.interpolate(low_aligned, size=high_res.shape[-2:],
                              mode='bilinear', align_corners=False)
        low_up = self.norm(low_up)
        # sigmoid → [0,1] for per channel λ
        return high_res + torch.sigmoid(self.weight) * low_up

# -----------------------------------------------------------------------------
# PGI– Programmable Gradient Injection
# -----------------------------------------------------------------------------


class PGIModule(nn.Module):
    """
    Programmable Gradient Injection v2 – DDP‑safe
    * If aux_channels is provided, a 1x1 ‘aux_adapter’ is set up during initialization (static, DDP‑safe).
    * The auxiliary contribution is only added in training mode.
    """
    def __init__(
        self,
        channels: int,
        reduction: int = 4,
        lambda_bounds: tuple[float, float] = (0.2, 0.7),
        init_lambda: float = 0.5,
        use_bn: bool = True,
        drop_path: float = 0.0,
        norm_factory: NormFactory = NormFactory("gn"),
        aux_channels: int | None = None,   # ◀︎ yeni
    ) -> None:
        super().__init__()
        self.channels = channels
        self.lambda_min, self.lambda_max = lambda_bounds

        # main branch
        self.main = nn.Sequential(
            nn.Conv2d(channels, channels, 3, 1, 1,
                      groups=max(1, channels // 4), bias=False),
            norm_factory(channels) if use_bn else nn.Identity(),
        )

        mid = max(channels // reduction, 32)
        self.aux = nn.Sequential(
            nn.Conv2d(channels, mid, 1, bias=False),
            norm_factory(mid) if use_bn else nn.Identity(),
            nn.SiLU(inplace=True),
            nn.Conv2d(mid, channels, 1, bias=False),
            norm_factory(channels) if use_bn else nn.Identity(),
        )

        # Aux channel adapter (static)
        if aux_channels is None or aux_channels == channels:
            self.aux_adapter = nn.Identity()
        else:
            self.aux_adapter = nn.Conv2d(aux_channels, channels, 1, bias=False)
            nn.init.kaiming_normal_(self.aux_adapter.weight, mode='fan_out', nonlinearity='relu')

        # λ (learnable)
        import math
        init_logit = math.log((init_lambda - self.lambda_min) / (self.lambda_max - init_lambda))
        self._lambda_logit = nn.Parameter(torch.tensor(init_logit))

        # Gate start 0.6
        self.gate = nn.Parameter(torch.ones(1, channels, 1, 1) * 0.5)

        self.dp = DropPath(drop_path) if drop_path > 0.0 else nn.Identity()
        self._init_weights()

    @property
    def lambda_val(self) -> torch.Tensor:
        σ = torch.sigmoid(self._lambda_logit)
        return self.lambda_min + (self.lambda_max - self.lambda_min) * σ

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight); nn.init.zeros_(m.bias)
            elif isinstance(m, nn.LayerNorm):
                if hasattr(m, 'weight'): nn.init.ones_(m.weight)
                if hasattr(m, 'bias'):   nn.init.zeros_(m.bias)

    def _align_spatial(self, feat: torch.Tensor, target_hw: tuple[int, int]) -> torch.Tensor:
        if feat.shape[-2:] != target_hw:
            feat = F.interpolate(feat, size=target_hw, mode='bilinear', align_corners=False)
        return feat

    def forward(self, x: torch.Tensor, aux_input: torch.Tensor | None = None):
        main_out = self.main(x)

        if aux_input is not None and self.training:
            aux = self.aux_adapter(aux_input)
            aux = self._align_spatial(aux, x.shape[-2:])
            aux = self.aux(aux) * self.gate
            main_out = main_out + self.lambda_val * aux

        return x + self.dp(main_out)

def get_drop_path_rates(num_layers: int, max_rate: float) -> List[float]:
    """Generate drop path rates with cosine scheduling

    Args:
        num_layers: Total number of layers
        max_rate: Maximum drop path rate

    Returns:
        List of drop path rates for each layer
    """
    if num_layers <= 1:
        return [0.0] * num_layers

    # Cosine scheduling for smooth progression
    rates = [
        max_rate * (1.0 - math.cos(math.pi * i / (num_layers - 1))) * 0.5
        for i in range(num_layers)
    ]

    return rates


def init_weights_improved(model: nn.Module):
    """Improved weight initialization for all module types"""
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d):
            # Special handling for different conv types
            if 'dw' in name or module.groups > 1:  # Depthwise
                nn.init.kaiming_normal_(module.weight, mode='fan_in', nonlinearity='linear')
            elif 'pw' in name or module.kernel_size == (1, 1):  # Pointwise
                nn.init.kaiming_normal_(module.weight, mode='fan_out', nonlinearity='relu')
            else:  # Regular conv
                nn.init.kaiming_normal_(module.weight, mode='fan_out', nonlinearity='relu')

            if module.bias is not None:
                nn.init.zeros_(module.bias)

        elif isinstance(module, nn.BatchNorm2d):
            nn.init.ones_(module.weight)
            nn.init.zeros_(module.bias)

        elif isinstance(module, LayerNormProxy):
            if hasattr(module, 'weight'):
                nn.init.ones_(module.weight)
            if hasattr(module, 'bias'):
                nn.init.zeros_(module.bias)

        elif isinstance(module, nn.Linear):
            nn.init.trunc_normal_(module.weight, std=0.02)
            if module.bias is not None:
                nn.init.zeros_(module.bias)

        elif isinstance(module, nn.Embedding):
            nn.init.normal_(module.weight, std=0.02)


# -----------------------------------------------------------------------------
# 4. Channel-wise Attention (SE)
# -----------------------------------------------------------------------------
class ChannelAttention(nn.Module):

    def __init__(self, channels: int, reduction: int = 8):
        super().__init__()
        mid = max(channels // reduction, 16)

        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Conv2d(channels, mid, 1, bias=True)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Conv2d(mid, channels, 1, bias=True)
        self.sigmoid = nn.Sigmoid()

        # --- parameters
        nn.init.kaiming_normal_(self.fc1.weight, mode='fan_out', nonlinearity='relu')
        nn.init.zeros_(self.fc1.bias)

        nn.init.kaiming_normal_(self.fc2.weight, mode='fan_out', nonlinearity='sigmoid')
        nn.init.zeros_(self.fc2.bias)

        # ►–– init scale ≈
        self.residual_scale = nn.Parameter(torch.tensor(-2.0))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        y = self.avgpool(x)
        y = self.relu(self.fc1(y))
        y = self.sigmoid(self.fc2(y))

        scale = torch.sigmoid(self.residual_scale)   # ∈(0,1)
        return x * (1 - scale) + x * y * scale
# 5. Gradient-clip helpers
# -----------------------------------------------------------------------------

def clip_dcnv4_grads(model: nn.Module, max_norm: float = 0.2) -> None:
    """Clip gradients for DCNv4 offset/mask parameters - more aggressive"""
    target_params = []
    for name, param in model.named_parameters():
        if param.grad is None:
            continue
        n_low = name.lower()
        if "offset" in n_low or "mask" in n_low:
            target_params.append(param)

    if target_params:
        torch.nn.utils.clip_grad_norm_(target_params, max_norm=max_norm)

# -----------------------------------------------------------------------------
# Helper classes
# -----------------------------------------------------------------------------

def fix_zero_init_params(model):
    """Fix parameters that are initialized to zero"""
    for name, param in model.named_parameters():
        if param.dim() > 0 and torch.all(param == 0):
            print(f"Fixing zero-initialized parameter: {name}")
            if 'bias' in name:
                nn.init.constant_(param, 0.01)
            else:
                nn.init.normal_(param, std=0.01)

def apply_gradient_checkpointing(model):
    """Apply gradient checkpointing to backbone stages - IMPROVED"""
    if hasattr(model, 'backbone'):
        # Apply a checkpoint for each stage
        for stage_name in ['st0', 'st1', 'st2', 'st3']:
            if hasattr(model.backbone, stage_name):
                stage = getattr(model.backbone, stage_name)
                # Make the sequential module checkpoint-friendly
                class CheckpointedSequential(nn.Module):
                    def __init__(self, *layers):
                        super().__init__()
                        self.layers = nn.ModuleList(layers)

                    def forward(self, x):
                        for layer in self.layers:
                            x = checkpoint(layer, x, use_reentrant=False)
                        return x

                # Recreate Stage
                checkpointed = CheckpointedSequential(*[block for block in stage])
                setattr(model.backbone, stage_name, checkpointed)

def init_conv(layer: nn.Conv2d, fan_out: bool = True):
    """Kaiming normal initialization"""
    nn.init.kaiming_normal_(
        layer.weight,
        mode="fan_out" if fan_out else "fan_in",
        nonlinearity="relu",
    )
    if layer.bias is not None:
        nn.init.zeros_(layer.bias)


class LNAct(nn.Module):
    """LayerNormProxy + SiLU"""
    def __init__(self, channels: int):
        super().__init__()
        self.norm = LayerNormProxy(channels)
        self.act = nn.SiLU(inplace=True)

    def forward(self, x):
        return self.act(self.norm(x))


class ConvLNAct(nn.Sequential):
    """Conv + LayerNorm + Act fusion"""
    def __init__(self, in_ch: int, out_ch: int, k: int = 3,
                s: int = 1, p: int = None, g: int = 1):
        p = p if p is not None else k // 2
        super().__init__(
            nn.Conv2d(in_ch, out_ch, k, s, p, groups=g, bias=False),
            LNAct(out_ch)
        )
        nn.init.kaiming_normal_(self[0].weight, mode="fan_out", nonlinearity="relu")


# -----------------------------------------------------------------------------
# LayerScale
# -----------------------------------------------------------------------------
class LayerScale(nn.Module):
    """
    Learnable scalar per layer (γ) – Broadcast-Safe version.
    Works with both (B,C,H,W) and (B,N,C) tensor layouts.
    """
    def __init__(
        self,
        dim: int,
        inplace: bool = False,
        init_values: float = 1.0,
        depth: int | None = None,
    ):
        super().__init__()
        self.inplace = inplace
        if depth is not None:
            init_values = min(1.0, 0.5 * depth / 3)
        self.weight = nn.Parameter(torch.ones(dim) * init_values)

    def _shape_for(self, x: torch.Tensor) -> tuple[int, ...]:
        """
        Determines the shape to broadcast the weight based on x's placement.
        Priority: channel-end (… , C) → channel-first (B, C, …) → singular cases.
        """
        C = self.weight.numel()
        nd = x.dim()

        # 4D  image: (B, C, H, W) veya (B, H, W, C)
        if nd == 4:
            if x.shape[1] == C:          # (B, C, H, W)
                return (1, C, 1, 1)
            if x.shape[-1] == C:         # (B, H, W, C) - nadir
                return (1, 1, 1, C)

        # 3D: (B, N, C) or (B, C, N) or (C, H, W)
        if nd == 3:
            if x.shape[-1] == C:         # (B, N, C)
                return (1, 1, C)
            if x.shape[1] == C:          # (B, C, N)
                return (1, C, 1)
            if x.shape[0] == C:          # (C, H, W) / (C, N, ?)such as extreme cases
                return (C, 1, 1)

        # 2D: (B, C) veya (C, B)
        if nd == 2:
            if x.shape[-1] == C:         # (B, C)
                return (1, C)
            if x.shape[0] == C:          # (C, B)
                return (C, 1)


        shape = [1] * max(1, nd)
        shape[-1] = C
        return tuple(shape)

    def forward(self, x: torch.Tensor):
        w = self.weight.view(*self._shape_for(x))
        if self.inplace:
            return x.mul_(w)
        return x * w

# Synchronize the LayerScale within DAT with this version (old behavior is preserved)
dat_mod.LayerScale = LayerScale


# -----------------------------------------------------------------------------
# Stem
# -----------------------------------------------------------------------------

class Stem(nn.Module):
    """Three 3×3 convolutions with stride pattern (2,1,2)"""
    def __init__(self, out_channels: int = 128):
        super().__init__()
        c_mid = out_channels // 2

        self.conv1 = nn.Conv2d(3, c_mid, kernel_size=3, stride=2, padding=1, bias=False)
        self.conv2 = nn.Conv2d(c_mid, c_mid, kernel_size=3, stride=1, padding=1, bias=False)
        self.conv3 = nn.Conv2d(c_mid, out_channels, kernel_size=3, stride=2, padding=1, bias=False)

        self.norm = LayerNormProxy(out_channels)
        self.act = nn.GELU()

        self._init_weights()

    def _init_weights(self):
        for m in [self.conv1, self.conv2, self.conv3]:
            nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.act(self.conv1(x))
        x = self.act(self.conv2(x))
        x = self.conv3(x)
        x = self.norm(x)
        return x


# -----------------------------------------------------------------------------
# DCNv4Lite
# -----------------------------------------------------------------------------


class DCNv4Lite(nn.Module):
    """
    • Fully wraps the DCNv4 path (2D→3D→2D), NO baseline conv/residual mix.
    • Offset/mask parameters are initialized with small std (for stable fp32 training).
    • Silently discards ‘mix_*’ and ‘conv.*’ keys in old checkpoints.

    Args:
        channels (int)            : Num of channels
        group (int)               : DCNv4 group
        kernel_size (int)         : Kernel
        stride (int)              : Stride
        pad (int)                 : Padding
        dilation (int)            : Dilation
        offset_scale (float)      : DCNv4 offset scale
        without_pointwise (bool)  : Enable/disable the internal 1x1 project in DCNv4 (default: True)
        init_offset_mask_std (float): offset/mask start std (default: 0.01)
        **kwargs                  : It is forwarded to DCNv4 exactly as is.
    """
    def __init__(self,
                 channels: int,
                 group: int = 8,
                 kernel_size: int = 3,
                 stride: int = 1,
                 pad: int = 1,
                 dilation: int = 1,
                 offset_scale: float = 0.1,
                 *,
                 without_pointwise: bool = True,
                 init_offset_mask_std: float = 0.01,
                 **kwargs):
        super().__init__()
        self.channels = int(channels)

        #  DCNv4
        self.dcn = DCNv4(
            channels=channels, kernel_size=kernel_size, stride=stride, pad=pad,
            dilation=dilation, group=group, offset_scale=offset_scale,
            center_feature_scale=False, remove_center=False,
            output_bias=True, without_pointwise=without_pointwise, **kwargs
        )

        # Offset/Mask secure init
        self._init_dcn_params(std=init_offset_mask_std)

        # Save geometry for representation (debug)
        self._geom = dict(k=kernel_size, s=stride, p=pad, d=dilation, g=group,
                          wop=without_pointwise)

    # ---------------- init helpers ----------------
    def _init_dcn_params(self, std: float = 0.01):
        """
        Offset/mask parameters are initialized with small normals so that
        sigmoid(0)=0.5 exits the plateau lock; other parameters remain in DCNv4's
        default init.
        """
        std = float(max(std, 1e-5))
        for n, p in self.dcn.named_parameters():
            n_low = n.lower()
            if ('offset' in n_low) or ('mask' in n_low):
                nn.init.normal_(p, std=std)

    # ---------------- forward ----------------------
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: (B, C, H, W) → y: (B, C, H, W)
        The DCNv4 interface (B, HW, C) requires a 2D→3D→2D conversion.
        """
        B, C, H, W = x.shape
        x3d = x.permute(0, 2, 3, 1).contiguous().view(B, H * W, C)   # (B, HW, C)
        y3d = self.dcn(x3d, shape=(H, W))                            # (B, HW, C)
        y   = y3d.view(B, H, W, C).permute(0, 3, 1, 2).contiguous()  # (B, C, H, W)
        return y

    # --------------- helpers -------------------
    @torch.no_grad()
    def freeze_offsets(self, flag: bool = True):
        """Freeze/unfreeze offset/mask parameters (e.g., for warming)."""
        for n, p in self.dcn.named_parameters():
            n_low = n.lower()
            if ('offset' in n_low) or ('mask' in n_low):
                p.requires_grad = (not flag)

    def offset_mask_parameters(self):
        """In the Optimizer, it yields the offset/mask parameters to provide separate LR/clip."""
        for n, p in self.dcn.named_parameters():
            n_low = n.lower()
            if ('offset' in n_low) or ('mask' in n_low):
                yield p

    # Ignore the mix/baseline keys remaining in old checkpoints
    def _load_from_state_dict(self, state_dict, prefix, *args, **kwargs):
        obsolete = (
            "mix_weight", "mix_logit", "_mix_tau", "_mix_eps", "_mix_clamp",
            "conv.weight", "conv.bias"
        )
        for key in list(state_dict.keys()):
            if key.startswith(prefix) and any(key.endswith(obs) for obs in obsolete):
                state_dict.pop(key)
        return super()._load_from_state_dict(state_dict, prefix, *args, **kwargs)

    def extra_repr(self) -> str:
        g = self._geom
        return (f"channels={self.channels}, k={g['k']}, s={g['s']}, p={g['p']}, "
                f"d={g['d']}, group={g['g']}, without_pointwise={g['wop']}")


def boost_relative_position_bias(model: nn.Module, std: float = 0.02):
    """
    Resets all `relative_position_bias_table` parameters in all ShiftWindow/LocalAttention modules belonging to the model.
    """
    for m in model.modules():
        if hasattr(m, "relative_position_bias_table"):
            nn.init.trunc_normal_(m.relative_position_bias_table, std=std)

def init_decoder_sampling_offsets(decoder: nn.Module, bias_val: float = 0.1):
    """
    Set the sampling_offsets bias in the deformable decoder layers
    to a fixed value; a small shift triggers the gradient.
    """
    for n, p in decoder.named_parameters():
        if "sampling_offsets.bias" in n:
            nn.init.constant_(p, bias_val)


def apply_hybrid_fixup(model: nn.Module):
    """
    a) Relative-pos bias table
    b) Decoder sampling_offsets bias
    c) (Optional) DCNv4 offset/mask gradient LR boost
    """
    boost_relative_position_bias(model)
    if hasattr(model, "decoder"):
        init_decoder_sampling_offsets(model.decoder, 0.1)
# -----------------------------------------------------------------------------
# Stage-0 Block
# -----------------------------------------------------------------------------

class StageZeroBlock(nn.Module):
    def __init__(self, channels=128, drop_path_rate=0.0,
                 use_layer_scale=True, layer_scale_init=1.0,
                 norm_factory=NormFactory("gn")):
        super().__init__()
        self.conv3 = nn.Conv2d(channels, channels, 3, 1, 1, bias=False)
        nn.init.kaiming_normal_(self.conv3.weight, mode='fan_out', nonlinearity='relu')

        self.se   = ChannelAttention(channels, reduction=8)
        self.norm = norm_factory(channels)
        self.scale = LayerScale(channels, init_values=layer_scale_init) if use_layer_scale else nn.Identity()
        self.dp = DropPath(drop_path_rate) if drop_path_rate > 0.0 else nn.Identity()

    def forward(self, x):
        res = x
        out = self.conv3(x)
        out = self.se(out)
        out = self.norm(out)
        out = self.scale(out)
        out = self.dp(out)
        return res + out

# -----------------------------------------------------------------------------
# Stage Blocks
# -----------------------------------------------------------------------------

class StageOneBlock(nn.Module):
    """
    DCNv4Lite + SE + (optional) LayerScale + DropPath
    """
    def __init__(self,
                channels: int = 128,
                drop_path: float = 0.0,
                use_layer_scale: bool = True,
                layer_scale_init: float = 1.0,
                norm_factory: NormFactory = NormFactory("gn")):
        super().__init__()
        self.dcn = DCNv4Lite(channels, group=8)
        self.se  = ChannelAttention(channels, reduction=8)
        self.post_norm = norm_factory(channels)

        self.scale = (LayerScale(channels, init_values=layer_scale_init)
                      if use_layer_scale else nn.Identity())

        self.dp = DropPath(drop_path) if drop_path > 0.0 else nn.Identity()

    def forward(self, x):
        res = x
        out = self.dcn(x)
        out = self.se(out)
        out = self.post_norm(out)
        out = self.scale(out)
        out = self.dp(out)
        return res + out

class DownABlock(nn.Module):
    """
    Depthwise‑Conv (s=2)  +  Pointwise‑Conv  +  Norm + Act
    """
    def __init__(self,
                in_ch: int = 128,
                out_ch: int = 256,
                norm_factory: NormFactory = NormFactory("gn")):
        super().__init__()

        # 1) Depth‑wise strided conv
        self.dw = nn.Conv2d(in_ch, in_ch, 3, stride=2, padding=1,
                            groups=in_ch, bias=False)

        # 2) Point‑wise projection
        self.pw = nn.Conv2d(in_ch, out_ch, 1, bias=False)

        # 3) Normalization + Activation
        #    a) If the norm type is LayerNormProxy, use LNAct
        #    b) In other cases, use norm + SiLU
        nf_layer = norm_factory(out_ch)
        if isinstance(nf_layer, LayerNormProxy):
            # LNAct = LayerNormProxy + SiLU
            from DAT.models.dat_blocks import LNAct as _LNAct
            self.norm_act = _LNAct(out_ch)
        else:
            self.norm_act = nn.Sequential(nf_layer, nn.SiLU(inplace=True))

        # --- Kaiming init ---
        nn.init.kaiming_normal_(self.dw.weight, mode="fan_out", nonlinearity="relu")
        nn.init.kaiming_normal_(self.pw.weight, mode="fan_out", nonlinearity="relu")

    # --------------------------------------------------
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.dw(x)
        x = self.pw(x)
        return self.norm_act(x)


class StageTwoBlock(nn.Module):
    """
    Stage‑2: Only DCNv4Lite path (no DAT, no GateFuse).
    CSP split is kept for comparable FLOPs/latency.
    """
    def __init__(self,
                 channels: int = 256,
                 dcn_group: int = 16,
                 drop_path: float = 0.1,
                 norm_factory: NormFactory = NormFactory("gn")):
        super().__init__()
        assert channels % 2 == 0, "StageTwoBlock expects even channel count."
        self.split_channels = channels // 2

        # --- DCNv4 branch (only this one active) ---
        self.dcn = DCNv4Lite(self.split_channels, group=max(1, dcn_group // 2))
        self.post_norm = norm_factory(self.split_channels)
        self.post_act  = nn.SiLU(inplace=True)

        # --- Skip proj (for grad out) ---
        self.skip_proj = nn.Sequential(
            nn.Conv2d(self.split_channels, self.split_channels, 1, bias=False),
            norm_factory(self.split_channels)
        )

        # --- Output merging ---
        self.fusion = nn.Conv2d(channels, channels, 1, bias=False)
        self.ln     = norm_factory(channels)
        self.act    = nn.SiLU(inplace=True)
        self.dp     = DropPath(drop_path) if drop_path > 0.0 else nn.Identity()

        # Init
        nn.init.kaiming_normal_(self.fusion.weight, mode="fan_out", nonlinearity="relu")
        nn.init.kaiming_normal_(self.skip_proj[0].weight, mode="fan_out", nonlinearity="relu")

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # CSP split
        x1, x2 = torch.split(x, self.split_channels, dim=1)

        # Skip branch
        x1 = self.skip_proj(x1)

        # DCNv4 branch
        x2 = self.dcn(x2)
        x2 = self.post_act(self.post_norm(x2))

        # Fuse & residual
        out = torch.cat([x1, x2], dim=1)
        out = self.fusion(out)
        out = self.act(self.ln(out))
        return x + self.dp(out)


class DownBBlock(nn.Module):
    """DW 3×3 s2 → PW 1×1, 256c → 640c"""
    def __init__(self, in_ch: int = 256, out_ch: int = 640,norm_factory: NormFactory = NormFactory("gn")):
        super().__init__()
        self.dw = nn.Conv2d(in_ch, in_ch, 3, stride=2, padding=1,
                          groups=in_ch, bias=False)
        self.pw = nn.Conv2d(in_ch, out_ch, 1, bias=False)
        self.norm = norm_factory(out_ch)
        self.act = nn.SiLU(inplace=True)

        for m in [self.dw, self.pw]:
            nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.act(self.norm(self.pw(self.dw(x))))


class StageThreeBlock(nn.Module):
    """Double DATHubLite + ChannelAttention"""
    def __init__(self, in_ch: int = 640, out_ch: int = 640, drop_path: float = 0.15,norm_factory: NormFactory = NormFactory("gn")):
        super().__init__()
        self.proj = nn.Conv2d(in_ch, out_ch, 1, bias=False)
        self.norm = norm_factory(out_ch)
        self.act = nn.SiLU(inplace=True)

        heads = max(4, out_ch // 64)
        self.dat1 = DATBlock(out_ch, heads=heads)
        self.dat2 = DATBlock(out_ch, heads=heads)
        self.dp1 = DropPath(drop_path)
        self.dp2 = DropPath(drop_path)
        self.ca = ChannelAttention(out_ch, reduction=8)

        nn.init.kaiming_normal_(self.proj.weight, mode="fan_out", nonlinearity="relu")

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.act(self.norm(self.proj(x)))
        res = x
        x = self.dat1(x)
        x = self.dp1(x) + res
        res = x
        x = self.dat2(x)
        x = self.dp2(x) + res
        x = self.ca(x)
        return x


class DownCBlock(nn.Module):
    """DW 3×3 s2 → PW 1×1, 640c → 768c"""
    def __init__(self, in_ch: int = 640, out_ch: int = 768,norm_factory: NormFactory = NormFactory("gn")):
        super().__init__()
        self.dw = nn.Conv2d(in_ch, in_ch, 3, stride=2, padding=1,
                          groups=in_ch, bias=False)
        self.pw = nn.Conv2d(in_ch, out_ch, 1, bias=False)
        self.norm = norm_factory(out_ch)
        self.act = nn.SiLU(inplace=True)

        nn.init.kaiming_normal_(self.dw.weight, mode="fan_out", nonlinearity="relu")
        nn.init.kaiming_normal_(self.pw.weight, mode="fan_out", nonlinearity="relu")

    def forward(self, x):
        x = self.dw(x)
        x = self.pw(x)

        return self.act(self.norm(x))
# -----------------------------------------------------------------------------
# StageAwareBackbone
# -----------------------------------------------------------------------------

class StageAwareBackbone(nn.Module):
    """
    Four-tier backbone based on DAT + DCN
    """
    def __init__(self,
                depths: list[int] = (3, 4, 8, 3),
                drop_path_max: float = 0.0,
                num_classes: int = 80,
                voc_prior: bool = False,
                norm_factory: NormFactory = NormFactory("gn"),
                use_layer_scale: bool = True,
                layer_scale_init: float = 1.0):
        super().__init__()
        self.num_classes = num_classes
        self.voc_prior   = voc_prior

        # ---- DropPath rates ----
        total_blocks = sum(depths)
        dpr = get_drop_path_rates(total_blocks, drop_path_max)
        for i in range(depths[0] + depths[1]):   # st0, st1
            dpr[i] = 0.0

        # ---- Stem ----
        self.stem = Stem(128)

        # ---- Stage‑0 ----
        dp_idx = 0
        self.st0 = nn.Sequential(*[
          StageZeroBlock(
              128,
              drop_path_rate=dpr[i],
              use_layer_scale=use_layer_scale,
              layer_scale_init=layer_scale_init,
              norm_factory=norm_factory
          )
          for i in range(depths[0])
      ])
        dp_idx += depths[0]

        # ---- Stage‑1 ----
        self.st1 = nn.Sequential(*[
            StageOneBlock(
                128,
                drop_path=dpr[dp_idx + i],
                use_layer_scale=use_layer_scale,
                layer_scale_init=layer_scale_init,
                norm_factory=norm_factory)
            for i in range(depths[1])
        ])
        dp_idx += depths[1]

        # ---- PGI + Down/CSI ----
        self.pgi_s1 = PGIModule(128, norm_factory=norm_factory, aux_channels=128)   # s0_out → s1_raw
        self.da     = DownABlock(128, 256, norm_factory=norm_factory)
        self.csi_s1_s2 = CrossScaleInjection(low_ch=128, high_ch=256)
        self.pgi_s2 = PGIModule(256, norm_factory=norm_factory, aux_channels=128)

        # ---- Stage‑2 ----
        self.st2 = nn.Sequential(*[
            StageTwoBlock(
                256, dcn_group=16,
                drop_path=dpr[dp_idx + i],
                norm_factory=norm_factory)
            for i in range(depths[2])
        ])
        dp_idx += depths[2]

        # ---- DownB / PGI ----
        self.db     = DownBBlock(256, 640, norm_factory=norm_factory)
        self.pgi_s3 = PGIModule(640, norm_factory=norm_factory, aux_channels=256)

        # ---- Stage‑3 ----
        self.st3 = nn.Sequential(*[
            StageThreeBlock(
                640, 640,
                drop_path=dpr[dp_idx + i],
                norm_factory=norm_factory)
            for i in range(depths[3])
        ])

        # ---- DownC ----
        self.dc = DownCBlock(640, 768, norm_factory=norm_factory)

        # ---- Auxiliary dense heads ----
        self.aux_head_s1 = nn.Conv2d(128, num_classes + 4, 1, bias=True)
        self.aux_head_s2 = nn.Conv2d(256, num_classes + 4, 1, bias=True)
        self.aux_head_s3 = nn.Conv2d(640, num_classes + 4, 1, bias=True)
        self._init_aux_heads()

    # --------------------------------------------------
    def _init_aux_heads(self):
        prior_value = 19 if self.voc_prior else 99
        for head in (self.aux_head_s1, self.aux_head_s2, self.aux_head_s3):
            nn.init.normal_(head.weight, std=0.01)
            nn.init.constant_(head.bias[: self.num_classes], -math.log(prior_value))
            nn.init.constant_(head.bias[self.num_classes :], 0.0)

    # --------------------------------------------------
    def forward(self, x: torch.Tensor, need_aux: bool = False):
        x      = self.stem(x)
        s0_out = self.st0(x)

        s1_raw = self.st1(s0_out)
        p3     = self.pgi_s1(s1_raw, aux_input=s0_out)

        p3_down = self.da(p3)
        s2_in   = self.csi_s1_s2(p3, p3_down)
        s2_in   = self.pgi_s2(s2_in, aux_input=p3)

        p4 = self.st2(s2_in)
        p4_down = self.db(p4)
        s3_in   = self.pgi_s3(p4_down, aux_input=p4)

        p5 = self.st3(s3_in)
        p6 = self.dc(p5)

        if self.training and need_aux:
            aux = {
                's1': self.aux_head_s1(p3),
                's2': self.aux_head_s2(p4),
                's3': self.aux_head_s3(p5),
            }
            return (p3, p4, p5, p6), aux
        return p3, p4, p5, p6






# ----------------------------------------------------------
# SpatialFuseNode
# ----------------------------------------------------------
class SpatialFuseNode(nn.Module):
    """
    Spatial-aware fuse (group-wise, production-ready, RPB-friendly)
      • Group-average energy map per input (B,G,K,H,W)
      • 1×1 projection per group (K→K), softplus-normalized
      • Initial fully uniform: proj.weight=0, bias=0  → equal w for each branch
      • Learnable temperature τ (with clamp), optional uniform floor (no dead branches)
      • No detach anywhere → gradients are unclipped (including RPBs)

    Args:
        n_inputs (int): K, how many features to combine (>=2)
        channels (int): C, number of channels
        groups (int): G, number of groups (automatically drops to 1 if C % G ≠ 0)
        tau_init (float): τ start (recommended: 0.9)
        learnable_tau (bool): Should τ be learned?
        eps (float): Numerical epsilon for normalization and splitting
        init_noise (float): (backward compatibility parameter; not used with uniform init)
        gate_floor (float): w ← (1-α)·w + α·(1/K); α∈[0,0.1] recommended, 0 closed
        tau_bounds (tuple): τ lower/upper bounds (e.g., (0.5, 2.0))
        uniform_init (bool): True ⇒ weight=0, bias=0 (full uniform initialization)
        save_last (bool): If True, last w is saved (for debug/regularizer)
        prenorm (str): “none” | “rms”  — RMS prenorm on the K axis (reduces saturation)

    Notes:
      • Returns extra_loss() for the entropy regularizer (optional, can be added to the loss).
      • The last w (B,G,K,H,W) can be inspected with get_last_weights() (if save_last=True).
    """
    def __init__(self,
                 n_inputs: int,
                 channels: int,
                 groups: int = 4,
                 tau_init: float = 0.9,
                 learnable_tau: bool = True,
                 eps: float = 5e-4,
                 init_noise: float = 1e-3,
                 *,
                 gate_floor: float = 0.0,
                 tau_bounds: tuple = (0.5, 2.0),
                 uniform_init: bool = True,
                 save_last: bool = False,
                 prenorm: str = "none"):
        super().__init__()
        assert n_inputs >= 2, "SpatialFuseNode: should be n_inputs >= 2 ."
        self.K = int(n_inputs)
        self.C = int(channels)
        self.G = int(groups) if (groups > 0 and channels % groups == 0) else 1
        self.gC = self.C // self.G
        self.eps = float(eps)
        self.gate_floor = float(gate_floor)
        self.tau_lo, self.tau_hi = float(tau_bounds[0]), float(tau_bounds[1])
        self.save_last = bool(save_last)
        self.prenorm = str(prenorm).lower()
        self._last_w = None  # debug amaçlı

        # Group-wise K→K projection (produces spatially-varying logit)
        # Takes input divided into G groups (B, G*K, H, W) → Output (B, G*K, H, W)
        self.proj = nn.Conv2d(self.G * self.K, self.G * self.K, kernel_size=1,
                              groups=self.G, bias=True)

        # --- Uniform start (to prevent early arm saturation) ---
        if uniform_init:
            nn.init.zeros_(self.proj.bias)
            with torch.no_grad():
                self.proj.weight.zero_()
        else:
            nn.init.kaiming_uniform_(self.proj.weight, a=math.sqrt(5))
            nn.init.zeros_(self.proj.bias)

        # --- temperature τ ---
        if learnable_tau:
            self.log_tau = nn.Parameter(torch.log(torch.tensor(float(tau_init))))
        else:
            self.register_buffer("log_tau", torch.log(torch.tensor(float(tau_init))), persistent=False)

    # ------------------ helpers ------------------
    @torch.no_grad()
    def set_tau(self, tau: float):
        """To adjust τ during heating."""
        v = max(1e-3, float(tau))
        t = torch.log(torch.tensor(v, device=self.log_tau.device, dtype=self.log_tau.dtype))
        self.log_tau.copy_(t)

    def get_last_weights(self):
        """The last calculated w (B, G, K, H, W). If save_last=True, it is saved."""
        return self._last_w

    def extra_loss(self) -> dict:
        """
        Optional regulator: gate entropy (higher values result in more balanced branching).
        You can add a small coefficient on the loss side (e.g., 1e-4..5e-4).
        """
        if self._last_w is None:
            return {}
        p = self._last_w.clamp_min(1e-8)
        # Entropy: -sum_k p log p / log(K)  → [0,1]
        ent = -(p * p.log()).sum(dim=2) / math.log(self.K)   # (B,G,H,W)
        return {"gate_entropy": ent.mean()}

    def _group_pool(self, x: torch.Tensor) -> torch.Tensor:
        # (B,C,H,W) → (B,G,H,W), average per group
        B, C, H, W = x.shape
        return x.view(B, self.G, self.gC, H, W).mean(dim=2)

    # ------------------ forward ------------------
    def forward(self, *features: torch.Tensor) -> torch.Tensor:
        # All inputs must be in the same format.
        K = len(features)
        assert K == self.K, f"SpatialFuseNode: {self.K} giriş bekleniyordu, {K} geldi."
        B, C, H, W = features[0].shape
        for f in features:
            assert f.shape == (B, C, H, W), "Tüm giriş feature'lar (B,C,H,W) aynı şekil olmalı."

        # 1) Grup havuzu → (B,G,K,H,W)
        gp = [self._group_pool(f) for f in features]
        gp_cat = torch.stack(gp, dim=2)  # (B,G,K,H,W)

        # 2) (optional) RMS prenorm → logit scale control on the K axis
        if self.prenorm == "rms":
            rms = gp_cat.pow(2).mean(dim=2, keepdim=True).add(1e-6).sqrt()
            gp_cat = gp_cat / rms

        # 3) Projection and gate logits
        x = gp_cat.flatten(1, 2)                      # (B, G*K, H, W)
        logits = self.proj(x).view(B, self.G, self.K, H, W)

        # 4) Softplus-normalize + τ
        tau = self.log_tau.exp().clamp(self.tau_lo, self.tau_hi)
        w = F.softplus(logits / tau) + 1e-9           # (B,G,K,H,W), her yerde >0
        w = w / (w.sum(dim=2, keepdim=True) + self.eps)

        # 5) Uniform floor (no dead leg, gradient flows to every leg)
        if self.gate_floor > 0.0:
            u = 1.0 / float(self.K)
            w = (1.0 - self.gate_floor) * w + self.gate_floor * u  # no need to normalize again

        if self.save_last:
            self._last_w = w.detach()

        # 6) Distribute weights to channels efficiently (no repeats, memory-friendly)
        #    w_k: (B,G,1,H,W), f: (B,G,gC,H,W) → contribution (B,C,H,W)
        out = None
        for k, f in enumerate(features):
            wk = w[:, :, k, :, :].unsqueeze(2)                 # (B,G,1,H,W)
            contrib = (f.view(B, self.G, self.gC, H, W) * wk).view(B, C, H, W)
            out = contrib if out is None else (out + contrib)

        return out


class RMSNorm2d(nn.Module):
    def __init__(self, eps: float = 1e-6):
        super().__init__()
        self.eps = float(eps)
    def forward(self, x):
        return x / (x.pow(2).mean(dim=(2,3), keepdim=True).add(self.eps).sqrt())

def _init_laplacian_dw(dw: nn.Conv2d):
    with torch.no_grad():
        k = torch.tensor([[0., 1., 0.],
                          [1.,-4., 1.],
                          [0., 1., 0.]], dtype=dw.weight.dtype, device=dw.weight.device)
        w = torch.zeros_like(dw.weight)
        w[:, 0, :, :] = k
        dw.weight.copy_(w)

def _ste_boost(x: torch.Tensor, gain: float) -> torch.Tensor:
    return x if gain <= 0 else (x + gain * (x - x.detach()))

class _FPNResBlock(nn.Module):
    def __init__(self, channels: int, drop_path: float = 0.0, norm_factory: 'NormFactory' = None):
        super().__init__()
        nf = norm_factory or NormFactory("gn")
        self.conv = ConvLNAct(channels, channels, k=3)
        self.ls   = LayerScale(channels, init_values=0.6)
        self.dp   = DropPath(drop_path) if drop_path > 0 else nn.Identity()
    def forward(self, x):
        return x + self.dp(self.ls(self.conv(x)))

class LightBiFPN(nn.Module):
    """
    LightBiFPN v4.1 — spatial‑aware, RPB‑friendly, repeat‑shared up/edge
      • SpatialFuseNode (group‑based 2D weights)
      • No‑blur in first top‑down (bilinear + norm/act)
      • Up-refine: DW 3×3 + (optional) Laplacian residual (small LS)
      • RMS prenorm (optional)
      • Low DropPath schedule (0.0 → 0.01)
      • Light grad-boost for p3
      • **New**: 3 up/edge blocks and **shared between iterations**
                  → No “NEVER UPDATED”, no parameter waste
    """
    def __init__(
        self,
        in_channels: tuple[int,int,int,int] = (128, 256, 640, 768),
        out: int = 256,
        repeats: int = 2,
        *,
        norm_factory: 'NormFactory' | None = None,
        # ---- fuse ----
        use_spatial_fuse: bool = True,
        fuse_groups: int = 4,
        fuse_tau_init: float = 0.9,
        fuse_learn_tau: bool = True,
        fuse_eps: float = 5e-4,
        init_noise: float = 1e-4,
        # ---- refine & norm ----
        prenorm: str = "rms",            # "none" | "rms"
        no_blur_first: bool = True,
        edge_enhance: bool = True,
        edge_ls_init: float = 0.03,
        # ---- drop path ----
        dp_top_second: float = 0.01,
        dp_bot_second: float = 0.01,
        # ---- grad boost ----
        grad_boost_low: float = 0.1,
    ):
        super().__init__()
        in3, in4, in5, in6 = in_channels
        self.repeats = int(repeats)
        self.out = int(out)
        self.norm_factory = norm_factory or NormFactory("gn")
        self.prenorm = prenorm.lower()
        self.no_blur_first = bool(no_blur_first)
        self.edge_enhance = bool(edge_enhance)
        self.grad_boost_low = float(grad_boost_low)
        self.use_spatial_fuse = bool(use_spatial_fuse)
        self.fuse_groups = int(fuse_groups)
        self.fuse_tau_init = float(fuse_tau_init)
        self.fuse_learn_tau = bool(fuse_learn_tau)
        self.fuse_eps = float(fuse_eps)
        self.init_noise = float(init_noise)

        # 1x1 projections
        self.p3_in = ConvLNAct(in3, out, k=1, p=0)
        self.p4_in = ConvLNAct(in4, out, k=1, p=0)
        self.p5_in = ConvLNAct(in5, out, k=1, p=0)
        self.p6_in = ConvLNAct(in6, out, k=1, p=0)

        # DropPath schedule
        top_dps = [0.0, float(dp_top_second)]
        bot_dps = [0.0, float(dp_bot_second)]
        self.top_convs = nn.ModuleList([
            _FPNResBlock(out, drop_path=top_dps[i // 3], norm_factory=self.norm_factory)
            for i in range(3 * self.repeats)
        ])
        self.bot_convs = nn.ModuleList([
            _FPNResBlock(out, drop_path=bot_dps[i // 4], norm_factory=self.norm_factory)
            for i in range(4 * self.repeats)
        ])

        # Fuse nodes
        def _make_fuse(K: int):
            from typing import Callable

            return SpatialFuseNode(
                                      K, channels=out, groups=self.fuse_groups,
                                      tau_init=1.4, learnable_tau=True,
                                      tau_bounds=(0.8, 1.8),
                                      prenorm="rms",
                                      gate_floor=0.02,          # no death
                                      uniform_init=True,
                                      eps=5e-4,
                                      save_last=True
                                  )

        self.fuse2_top = nn.ModuleList([_make_fuse(2) for _ in range(3 * self.repeats)])
        self.fuse2_bot = nn.ModuleList([_make_fuse(2) for _ in range(3 * self.repeats)])
        self.fuse3_bot = nn.ModuleList([_make_fuse(3) for _ in range(1 * self.repeats)])

        # ---  3 up/edge blocks and shared between repeats ---
        self.up_dw, self.up_na = nn.ModuleList(), nn.ModuleList()
        self.edge_dw, self.edge_ls = nn.ModuleList(), nn.ModuleList()
        for _ in range(3):  # only for 3 items
            dw = nn.Conv2d(out, out, 3, 1, 1, groups=out, bias=False)
            nn.init.kaiming_normal_(dw.weight, mode="fan_out", nonlinearity="relu")
            self.up_dw.append(dw)

            nf_layer = self.norm_factory(out)
            if isinstance(nf_layer, LayerNormProxy):
                from DAT.models.dat_blocks import LNAct as _LNAct
                self.up_na.append(_LNAct(out))
            else:
                self.up_na.append(nn.Sequential(nf_layer, nn.SiLU(inplace=True)))

            edw = nn.Conv2d(out, out, 3, 1, 1, groups=out, bias=False)
            _init_laplacian_dw(edw)
            self.edge_dw.append(edw)
            self.edge_ls.append(LayerScale(out, init_values=edge_ls_init))

        # High-level passthrough LS (p5, p4, p3)
        self.keep_top = nn.ModuleList([LayerScale(out, init_values=0.10) for _ in range(3)])

        # Input balancing
        self.balance = RMSNorm2d(eps=1e-6) if self.prenorm == "rms" else nn.Identity()

    # --- helpers ---
    def _up_refine_shared(self, x: torch.Tensor, size_hw: tuple[int,int], step_id: int, rep_idx: int) -> torch.Tensor:
        """
        step_id: [0,1,2] → corresponds to each top-down step, independent of repeat
        """
        x = F.interpolate(x, size=size_hw, mode="bilinear", align_corners=False)
        if self.no_blur_first and rep_idx == 0:
            # no-blur first repeat: only norm/act
            x = self.up_na[step_id](x)
        else:
            x = self.up_dw[step_id](x)
            if self.edge_enhance:
                x = x + self.edge_ls[step_id](self.edge_dw[step_id](x))
            x = self.up_na[step_id](x)
        return x

    def _bal(self, *xs):
        return (tuple(self.balance(x) for x in xs) if self.prenorm != "none" else xs)

    def forward(self, p3, p4, p5, p6):
        # Proj
        p3, p4, p5, p6 = self.p3_in(p3), self.p4_in(p4), self.p5_in(p5), self.p6_in(p6)

        # Low-level grad-boost
        if self.training and self.grad_boost_low > 0:
            p3 = _ste_boost(p3, self.grad_boost_low)

        tconv = bconv = f2t = f2b = f3b = 0

        for rep in range(self.repeats):
            # ---------- Top‑down ----------
            # step_id = 0: p6→p5
            u5 = self._up_refine_shared(p6, p5.shape[-2:], step_id=0, rep_idx=rep)
            p5_b, u5_b = self._bal(p5, u5)
            p5_td = self.fuse2_top[f2t](p5_b, u5_b); f2t += 1
            p5_td = self.top_convs[tconv](p5_td); tconv += 1
            p5_td = p5_td + self.keep_top[0](p5)

            # step_id = 1: p5_td→p4
            u4 = self._up_refine_shared(p5_td, p4.shape[-2:], step_id=1, rep_idx=rep)
            p4_b, u4_b = self._bal(p4, u4)
            p4_td = self.fuse2_top[f2t](p4_b, u4_b); f2t += 1
            p4_td = self.top_convs[tconv](p4_td); tconv += 1
            p4_td = p4_td + self.keep_top[1](p4)

            # step_id = 2: p4_td→p3
            u3 = self._up_refine_shared(p4_td, p3.shape[-2:], step_id=2, rep_idx=rep)
            p3_b, u3_b = self._bal(p3, u3)
            p3_td = self.fuse2_top[f2t](p3_b, u3_b); f2t += 1
            p3_td = self.top_convs[tconv](p3_td); tconv += 1
            p3_td = p3_td + self.keep_top[2](p3)

            # ---------- Bottom‑up ----------
            p3 = self.fuse2_bot[f2b](*self._bal(p3, p3_td)); f2b += 1
            p3 = self.bot_convs[bconv](p3); bconv += 1

            p4 = self.fuse2_bot[f2b](*self._bal(p4_td, F.max_pool2d(p3, 2))); f2b += 1
            p4 = self.bot_convs[bconv](p4); bconv += 1

            p5 = self.fuse3_bot[f3b](*self._bal(p5, F.max_pool2d(p4, 2), p5_td)); f3b += 1
            p5 = self.bot_convs[bconv](p5); bconv += 1

            p6 = self.fuse2_bot[f2b](*self._bal(p6, F.max_pool2d(p5, 2))); f2b += 1
            p6 = self.bot_convs[bconv](p6); bconv += 1

        return p3, p4, p5, p6


# -----------------------------------------------------------------------------
# Positional Encoding
# -----------------------------------------------------------------------------

class Pos2d(nn.Module):
    """2D sinusoidal positional encoding"""
    def __init__(self, c: int = 320, max_h: int = 640, max_w: int = 640):
        super().__init__()
        assert c % 4 == 0
        self.cq = c // 4
        pos = self._build(max_h, max_w, c)
        self.register_buffer("pos_table", pos, persistent=False)

    def _build(self, H: int, W: int, C: int) -> torch.Tensor:
        yv, xv = torch.meshgrid(
            torch.linspace(0, 1, H), torch.linspace(0, 1, W), indexing="ij"
        )
        div = torch.exp(torch.arange(0, self.cq) * (-math.log(10000.0) / self.cq))
        pos_x = (xv[..., None] * div).reshape(H, W, -1)
        pos_y = (yv[..., None] * div).reshape(H, W, -1)
        pos = torch.cat([pos_y.sin(), pos_y.cos(), pos_x.sin(), pos_x.cos()], dim=2)
        return pos.permute(2, 0, 1).unsqueeze(0)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        _, _, H, W = x.shape
        return self.pos_table[:, :, :H, :W]



import inspect

# -----------------------------------------------------------------------------
# DATBlock –  TransformerStage wrapper
# -----------------------------------------------------------------------------
class DATBlock(nn.Module):
    def __init__(self,
                channels: int,
                depth: int = 2,
                heads: int | None = None,
                window_size: int = 8,
                drop: float = 0.0):
        super().__init__()
        heads = heads or max(4, channels // 32)

        cfg = dict(
            fmap_size           = (window_size, window_size),
            window_size         = window_size,
            ns_per_pt           = 4,
            dim_in              = channels,
            dim_embed           = channels,
            depths              = depth,               # INT
            stage_spec          = 'N' * depth,
            n_groups            = 1,
            use_pe              = True,
            sr_ratio            = 1,
            heads               = heads,               # INT
            heads_q             = [heads] * depth,     # LIST
            stride              = 1,
            offset_range_factor = 2,
            dwc_pe              = True,
            no_off              = False,
            fixed_pe            = False,
            attn_drop           = drop,
            proj_drop           = drop,
            expansion           = 4,
            drop                = drop,
            drop_path_rate      = [drop] * depth,      # LIST
            use_dwc_mlp         = False,
            ksize               = 3,
            nat_ksize           = 7,
            k_qna               = 8,
            nq_qna              = 9,
            qna_activation      = "relu",
            layer_scale_value   = 0.3,
            use_lpu             = False,
            log_cpb             = True,
        )

        from DAT.models.dat import TransformerStage
        self.stage = TransformerStage(**cfg)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x = (B,C,H,W) –
        if hasattr(self.stage, 'fmap_size'):
            self.stage.fmap_size = x.shape[-2:]
        try:
            return self.stage(x)
        except TypeError:                           # some versions (x,H,W) require
            H, W = x.shape[-2:]
            return self.stage(x, H, W)



# -----------------------------------------------------------------------------
# Deformable Encoder
# -----------------------------------------------------------------------------

class GLUFFN(nn.Module):
    """
    GLU-based FFN:
      - First linear: d_model -> 2*inner
      - Gate: SwiGLU (SiLU) or GEGLU (GELU)
      - Second linear: inner -> d_model

    inner width:
      • if ffn_dim is given: inner = ffn_dim // 2  (2*inner = ffn_dim)
      • otherwise: inner = round(d_model * ffn_mult)
    """
    def __init__(
        self,
        d_model: int,
        ffn_dim: Optional[int] = None,
        ffn_mult: float = 2.0,
        act: str = "swiglu",
        drop: float = 0.0,
    ):
        super().__init__()
        if ffn_dim is not None:
            inner = max(32, int(ffn_dim) // 2)
        else:
            inner = max(32, int(round(d_model * float(ffn_mult))))
        self.fc1 = nn.Linear(d_model, inner * 2, bias=True)
        self.fc2 = nn.Linear(inner, d_model, bias=True)
        self.drop = nn.Dropout(drop) if drop > 0 else nn.Identity()

        act = act.lower()
        if act not in ("swiglu", "geglu"):
            raise ValueError("GLUFFN.act must be 'swiglu' or 'geglu'")
        self.act_kind = act

        nn.init.xavier_uniform_(self.fc1.weight); nn.init.zeros_(self.fc1.bias)
        nn.init.xavier_uniform_(self.fc2.weight); nn.init.zeros_(self.fc2.bias)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        xh = self.fc1(x)
        x_lin, x_gate = xh.chunk(2, dim=-1)
        if self.act_kind == "swiglu":
            gated = F.silu(x_gate) * x_lin
        else:  # 'geglu'
            gated = F.gelu(x_gate) * x_lin
        return self.drop(self.fc2(gated))




class DeformEncoderLayer(nn.Module):
    """
    MS-Deformable self-attention + GLU-FFN, Pre-Norm + LayerScale + DropPath.

    Args (backward-compatible):
        d_model, n_heads, n_levels, n_points
        ffn_dim:   optional (if given, GLU inner width is ffn_dim//2)
        ffn_mult:  used if ffn_dim is not specified (default 2.0)
        ffn_act:   ‘swiglu’ (default) or ‘geglu’
        drop, drop_path
    """
    def __init__(
        self,
        d_model: int = 320,
        n_heads: int = 10,
        n_levels: int = 4,
        n_points: int = 4,
        *,
        ffn_dim: Optional[int] = None,
        ffn_mult: float = 2.0,
        ffn_act: str = "swiglu",
        drop: float = 0.0,
        drop_path: float = 0.1,
    ):
        super().__init__()

        self.self_attn = MultiScaleDeformableAttention(
            embed_dims=d_model, num_heads=n_heads,
            num_levels=n_levels, num_points=n_points,
            batch_first=True,
        )

        self.norm1 = nn.LayerNorm(d_model)
        self.ls1   = LayerScale(d_model, init_values=0.4)
        self.drop1 = nn.Dropout(drop) if drop > 0 else nn.Identity()
        self.dp1   = DropPath(drop_path) if drop_path > 0 else nn.Identity()

        self.ffn   = GLUFFN(d_model, ffn_dim=ffn_dim, ffn_mult=ffn_mult, act=ffn_act, drop=drop)
        self.norm2 = nn.LayerNorm(d_model)
        self.ls2   = LayerScale(d_model, init_values=0.4)
        self.drop2 = nn.Dropout(drop) if drop > 0 else nn.Identity()
        self.dp2   = DropPath(drop_path) if drop_path > 0 else nn.Identity()

    @staticmethod
    def _make_encoder_reference_points(
        spatial_shapes: torch.Tensor, B: int, device, dtype
    ) -> torch.Tensor:
        ref_list = []
        for (H, W) in spatial_shapes.tolist():
            ref_y, ref_x = torch.meshgrid(
                torch.linspace(0.5, H - 0.5, H, device=device, dtype=dtype) / H,
                torch.linspace(0.5, W - 0.5, W, device=device, dtype=dtype) / W,
                indexing="ij",
            )
            ref = torch.stack((ref_x, ref_y), dim=-1).reshape(-1, 2)  # (HW,2)
            ref_list.append(ref)
        return torch.cat(ref_list, dim=0)[None, :, None, :].repeat(B, 1, spatial_shapes.size(0), 1)

    def forward(
        self,
        src: torch.Tensor,             # (B, sumHW, C)
        pos: torch.Tensor,             # (B, sumHW, C)
        spatial_shapes: torch.Tensor,  # (L, 2)
        lvl_start_idx: torch.Tensor,   # (L,)
        key_padding_mask: Optional[torch.Tensor] = None,
    ) -> torch.Tensor:

        B, N, C = src.shape
        ref_pts = self._make_encoder_reference_points(
            spatial_shapes=spatial_shapes, B=B, device=src.device, dtype=src.dtype
        )  # (B,N,L,2)

        x = src
        q = self.norm1(x)
        attn_out = self.self_attn(
            query=q, value=q,
            reference_points=ref_pts,
            spatial_shapes=spatial_shapes,
            level_start_index=lvl_start_idx,
            key_padding_mask=key_padding_mask,
            query_pos=pos,
        )
        x = x + self.dp1(self.ls1(self.drop1(attn_out)))

        y = self.ffn(self.norm2(x))
        x = x + self.dp2(self.ls2(self.drop2(y)))
        return x

class TinyDeformEncoder(nn.Module):
    """
    Two layers are recommended (lightweight and effective).
    get_drop_path_rates() is available;
    """
    def __init__(self,
                 num_layers: int = 1,
                 d_model: int = 320,
                 n_heads: int = 10,
                 n_levels: int = 4,
                 n_points: int = 4,
                 ffn_dim: int = 1024,
                 drop: float = 0.0,
                 drop_path_max: float = 0.1):
        super().__init__()
        dpr = get_drop_path_rates(num_layers, drop_path_max)
        self.layers = nn.ModuleList([
            DeformEncoderLayer(d_model=d_model,
                               n_heads=n_heads,
                               n_levels=n_levels,
                               n_points=n_points,
                               ffn_dim=ffn_dim,
                               drop=drop,
                               drop_path=dpr[i])
            for i in range(num_layers)
        ])

    def forward(self,
                src: torch.Tensor,             # (B, sumHW, C)
                pos: torch.Tensor,             # (B, sumHW, C)
                spatial_shapes: torch.Tensor,  # (L,2)
                lvl_start_idx: torch.Tensor,   # (L,)
                key_padding_mask: Optional[torch.Tensor] = None) -> torch.Tensor:
        for ly in self.layers:
            src = ly(src, pos, spatial_shapes, lvl_start_idx, key_padding_mask)
        return src


# -----------------------------------------------------------------------------
# RT-Deform Decoder (Fixed IoU-aware padding)
# -----------------------------------------------------------------------------


class DeformDecoderLayer(nn.Module):
    """
    Decoder:
      Pre-Norm + LayerScale + DropPath + GLU-FFN + tanh-bounded ref update.

    Args (backward-compatible):
        d_model, n_heads, n_levels, n_points
        ffn_dim:   optional (for GLU inner width, 2*inner = ffn_dim)
        ffn_mult:  used if ffn_dim is not specified
        ffn_act:   ‘swiglu’ | ‘geglu’
        drop, drop_path
        refine_scale: float or (lo,hi)  — you can provide layer_id/num_layers for the schedule
        grad_eps: very small nudge
    """
    def __init__(
        self,
        d_model: int = 320,
        n_heads: int = 10,
        n_levels: int = 4,
        n_points: int = 4,
        *,
        ffn_dim: Optional[int] = None,
        ffn_mult: float = 2.0,
        ffn_act: str = "swiglu",
        drop: float = 0.0,
        drop_path: float = 0.1,
        refine_scale: Union[float, Tuple[float, float]] = 0.5,
        layer_id: Optional[int] = None,
        num_layers: Optional[int] = None,
        grad_eps: float = 1e-4,
    ):
        super().__init__()
        self.grad_eps = float(grad_eps)

        # refine scale
        if isinstance(refine_scale, (tuple, list)):
            lo, hi = float(refine_scale[0]), float(refine_scale[1])
            if (num_layers is not None) and (layer_id is not None) and (num_layers > 1):
                t = float(layer_id) / float(num_layers - 1)
                self.refine_scale = lo + (hi - lo) * t
            else:
                self.refine_scale = 0.5 * (lo + hi)
        else:
            self.refine_scale = float(refine_scale)

        # Self-Attn (MHA) + Pre-Norm
        self.self_attn = nn.MultiheadAttention(
            embed_dim=d_model, num_heads=n_heads, batch_first=True
        )
        self.norm1 = nn.LayerNorm(d_model)
        self.ls1   = LayerScale(d_model, init_values=0.4)
        self.drop1 = nn.Dropout(drop) if drop > 0 else nn.Identity()
        self.dp1   = DropPath(drop_path) if drop_path > 0 else nn.Identity()

        # Cross-Attn (MS-Deformable) + Pre-Norm
        self.cross_attn = MultiScaleDeformableAttention(
            embed_dims=d_model, num_heads=n_heads,
            num_levels=n_levels, num_points=n_points,
            batch_first=True,
        )
        self.norm2 = nn.LayerNorm(d_model)
        self.ls2   = LayerScale(d_model, init_values=0.4)
        self.drop2 = nn.Dropout(drop) if drop > 0 else nn.Identity()
        self.dp2   = DropPath(drop_path) if drop_path > 0 else nn.Identity()

        # FFN (GLU) + Pre-Norm
        self.ffn   = GLUFFN(d_model, ffn_dim=ffn_dim, ffn_mult=ffn_mult, act=ffn_act, drop=drop)
        self.norm3 = nn.LayerNorm(d_model)
        self.ls3   = LayerScale(d_model, init_values=0.4)
        self.drop3 = nn.Dropout(drop) if drop > 0 else nn.Identity()
        self.dp3   = DropPath(drop_path) if drop_path > 0 else nn.Identity()

        # Ref delta head (tanh clamp)
        self.ref_mlp = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.GELU(),
            nn.Linear(d_model, 2),
        )
        for m in self.ref_mlp:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight); nn.init.zeros_(m.bias)

    @staticmethod
    def _inv_sigmoid(x: torch.Tensor, eps: float = 1e-6) -> torch.Tensor:
        x = x.clamp(eps, 1.0 - eps)
        return torch.log(x) - torch.log(1.0 - x)

    def forward(
        self,
        tgt: torch.Tensor,                 # (B, N, C)
        ref_pts: torch.Tensor,             # (B, N, L, 2)
        src: torch.Tensor,                 # (B, S, C)
        query_pos: torch.Tensor,           # (B, N, C)
        spatial_shapes: torch.Tensor,      # (L, 2)
        lvl_start_idx: torch.Tensor,       # (L,)
    ) -> Tuple[torch.Tensor, torch.Tensor]:

        x = tgt
        q = self.norm1(x)
        sa_out = self.self_attn(q, q, q, need_weights=False)[0]
        x = x + self.dp1(self.ls1(self.drop1(sa_out)))

        q = self.norm2(x)
        ca_out = self.cross_attn(
            query=q, value=src,
            reference_points=ref_pts,
            spatial_shapes=spatial_shapes,
            level_start_index=lvl_start_idx,
            query_pos=query_pos,
        )
        x = x + self.dp2(self.ls2(self.drop2(ca_out)))

        y = self.ffn(self.norm3(x))
        x = x + self.dp3(self.ls3(self.drop3(y)))

        # tanh-bounded iterative ref update
        delta = torch.tanh(self.ref_mlp(x)) * self.refine_scale      # (B,N,2)
        new_ref = torch.sigmoid(self._inv_sigmoid(ref_pts) + delta.unsqueeze(2).to(ref_pts.dtype))

        # tiny nudge
        x = x + self.grad_eps * delta.mean(dim=2, keepdim=True)
        return x, new_ref


class HeadPrep(nn.Module):
    def __init__(self, d_model: int, ls_init: float = 0.80, ln_eps: float = 1e-6):
        super().__init__()
        self.norm = nn.LayerNorm(d_model, eps=ln_eps)
        self.ls   = LayerScale(d_model, init_values=ls_init)
    def forward(self, x):  # x: (B, N, C)
        return self.ls(self.norm(x))

class RTDeformDecoder(nn.Module):
    """
    ABLATION — RT‑Deform Decoder (No IoU head, No Denoising)


    * Denoising (DN) queries and the associated label/box noise mechanism have been removed.
    * The interface signature is preserved; the dn_queries / use_iou_aware parameters are ignored for ablation.
    """
    def __init__(self,
                 num_obj_classes: int = 20,
                 include_background: bool = True,
                 *,
                 num_queries: int = 300,
                 num_layers: int = 4,
                 d_model: int = 320,
                 attn_n_heads: int = 10,
                 dn_queries: int = 0,                 # ignored
                 use_iou_aware: bool = False,         # ignored
                 # Encoder
                 use_encoder: bool = True,
                 encoder_layers: int = 1,
                 encoder_drop_path_max: float = 0.1,
                 # (The following are retained for interface compatibility, not used)
                 iou_k_ratio: float = 0.75,
                 mqs_enable: bool = True,             # seed/MQS mechanism is not present here
                 mqs_obj_ratio: float = 0.30,
                 mqs_grid_ratio: float = 0.20,
                 mqs_levels: Tuple[int, ...] = (1, 2, 3),
                 mqs_local_max_kernel: int = 3,
                 mqs_train_only: bool = False,
                 seed_enable: bool = True,
                 seed_alpha_obj_init: float = 0.55,
                 seed_alpha_grid_init: float = 0.4,
                 seed_mlp_expansion: float = 1.0,
                 use_proposals: bool = True,
                 proposal_topk: Optional[int] = None,
                 proposal_ratio: float = 0.70,
                 min_mqs: int = 60,
                 min_left_queries: int = 16,
                 emit_obj_logits_in_eval: bool = False):
        super().__init__()

        # Number of classes
        self.num_obj_classes = num_obj_classes
        self.include_background = include_background
        self.num_pred = num_obj_classes + int(include_background)

        self.num_queries = int(num_queries)
        self.num_layers  = int(num_layers)
        self.d_model     = int(d_model)
        self.use_encoder = bool(use_encoder)

        # Position and level embed
        self.query_pos  = nn.Embedding(self.num_queries, d_model)  # learned positional (N,C)
        self.query_feat = nn.Embedding(self.num_queries, d_model)  # learned content   (N,C)
        self.pos_embed   = Pos2d(d_model)
        self.level_embed = nn.Parameter(torch.randn(4, d_model))

        # Encoder (optional)
        if self.use_encoder:
            self.encoder = TinyDeformEncoder(
                num_layers=encoder_layers,
                d_model=d_model, n_heads=attn_n_heads,
                n_levels=4, n_points=4,
                ffn_dim=d_model * 4,
                drop=0.0,
                drop_path_max=encoder_drop_path_max
            )

        # Decoder
        n_points_list = [4] * (num_layers - 2) + [2, 2] if num_layers >= 2 else [4]
        self.layers = nn.ModuleList([
            DeformDecoderLayer(d_model=d_model,
                               n_heads=attn_n_heads,
                               n_levels=4,
                               n_points=n_points_list[i],
                               ffn_dim=d_model * 4,
                               refine_scale=(0.15, 0.45),
                               layer_id=i, num_layers=num_layers)
            for i in range(num_layers)
        ])

        # Head prep + classification / box heads
        self.head_prep = HeadPrep(d_model, ls_init=0.80, ln_eps=1e-6)
        self.cls_head  = nn.Linear(d_model, self.num_pred)
        self.box_head  = nn.Linear(d_model, 4)

        # Aux head'ler (except last layer)
        if self.num_layers > 1:
            self.aux_cls_heads = nn.ModuleList(nn.Linear(d_model, self.num_pred) for _ in range(self.num_layers - 1))
            self.aux_box_heads = nn.ModuleList(nn.Linear(d_model, 4)            for _ in range(self.num_layers - 1))
        else:
            self.aux_cls_heads = nn.ModuleList([])
            self.aux_box_heads = nn.ModuleList([])

        # Small MLP for initial reference
        self.ref_init = nn.Linear(d_model, 2)

        self._init_weights()

    # ------------------- init helpers -------------------
    def _init_weights(self):
        nn.init.normal_(self.query_pos.weight,  std=0.02)
        nn.init.normal_(self.query_feat.weight, std=0.02)
        nn.init.normal_(self.level_embed,       std=0.02)

        nn.init.xavier_uniform_(self.ref_init.weight); nn.init.zeros_(self.ref_init.bias)

        nn.init.xavier_uniform_(self.box_head.weight, gain=1.0); nn.init.zeros_(self.box_head.bias)
        nn.init.xavier_uniform_(self.cls_head.weight, gain=1.0)

        for c, b in zip(self.aux_cls_heads, self.aux_box_heads):
            nn.init.xavier_uniform_(c.weight, gain=1.0)
            nn.init.xavier_uniform_(b.weight, gain=1.0); nn.init.zeros_(b.bias)

        self.reset_class_bias()

    def reset_class_bias(self, object_prior: float = .2, no_object_bias: float = -2.):
        obj_bias = -math.log((1. - object_prior) / object_prior)
        with torch.no_grad():
            self.cls_head.bias[:] = 0.
            self.cls_head.bias[: self.num_obj_classes] = obj_bias
            if self.include_background:
                self.cls_head.bias[-1] = no_object_bias
            for aux in self.aux_cls_heads:
                aux.bias[:] = self.cls_head.bias

    # ------------------- forward -------------------
    def forward(self,
                p3: torch.Tensor, p4: torch.Tensor,
                p5: torch.Tensor, p6: torch.Tensor,
                targets: Optional[List[Dict]] = None
                ) -> Dict[str, torch.Tensor]:

        B = p3.size(0)
        device = p3.device
        feats = [p6, p5, p4, p3]  # from largest to smallest (B, C, H, W)
        L = len(feats)

        # Per‑level src & pos (B, ΣHW, C)
        src_list, pos_list = [], []
        spatial_shapes = torch.zeros(L, 2, dtype=torch.long, device=device)  # (L,2) = (H,W)

        for i, f in enumerate(feats):
            B_, C, H, W = f.shape
            pos_lvl = self.pos_embed(f).expand(B_, -1, -1, -1) + self.level_embed[i].view(1, -1, 1, 1)
            src_list.append(f.flatten(2).transpose(1, 2))              # (B, HW, C)
            pos_list.append(pos_lvl.flatten(2).transpose(1, 2))         # (B, HW, C)
            spatial_shapes[i, 0] = H
            spatial_shapes[i, 1] = W

        src = torch.cat(src_list, dim=1)                         # (B, ΣHW, C)
        pos = torch.cat(pos_list, dim=1).to(dtype=src.dtype)     # (B, ΣHW, C)

        # level start index: (L,)
        numel_per_level = spatial_shapes[:, 0] * spatial_shapes[:, 1]
        lvl_start_idx = torch.cat(
            [numel_per_level.new_zeros(1), numel_per_level.cumsum(0)[:-1]],
            dim=0
        )

        # Encoder (optional)
        memory = self.encoder(src, pos, spatial_shapes, lvl_start_idx) if self.use_encoder else src
        if pos.size(0) != memory.size(0):
            pos = pos.expand(memory.size(0), -1, -1)
        pos = pos.to(dtype=memory.dtype)

        # Learned queries
        content = self.query_feat.weight.unsqueeze(0).expand(B, -1, -1)  # (B,N,C)
        qpos    = self.query_pos.weight.unsqueeze(0).expand(B, -1, -1)   # (B,N,C)
        init_ref = torch.sigmoid(self.ref_init(qpos))                    # (B,N,2)
        ref_pts  = init_ref.unsqueeze(2).expand(-1, -1, 4, -1)           # (B,N,4,2)

        tgt = content
        aux_out = []

        # Decoder
        for lid, layer in enumerate(self.layers):
            tgt, ref_pts = layer(
                tgt=tgt, ref_pts=ref_pts, src=memory,
                query_pos=qpos, spatial_shapes=spatial_shapes, lvl_start_idx=lvl_start_idx
            )

            if self.training and lid < self.num_layers - 1:
                h_aux = self.head_prep(tgt)  # LN + LayerScale
                aux_logits = self.aux_cls_heads[lid](h_aux)
                aux_boxes  = self.aux_box_heads[lid](h_aux).sigmoid()
                aux_out.append({"pred_logits": aux_logits, "pred_boxes": aux_boxes})

        # Final heads
        h = self.head_prep(tgt)
        logits = self.cls_head(h)
        boxes  = self.box_head(h).sigmoid()

        out: Dict[str, torch.Tensor] = {"pred_logits": logits, "pred_boxes": boxes,
                                        "final_ref_pts": ref_pts.contiguous()}
        if aux_out and self.training:
            out["aux_outputs"] = aux_out
        return out
# -----------------------------------------------------------------------------
# MiniBackbone
# -----------------------------------------------------------------------------

class MiniBackbone(nn.Module):
    """Lightweight version with CSI and PGI"""
    def __init__(self, depths: List[int] = [6, 6, 12, 6],
                drop_path_max: float = 0.2,
                num_classes: int = 20):
        super().__init__()
        self.backbone = StageAwareBackbone(depths, drop_path_max, num_classes)

    def forward(self, x: torch.Tensor, need_aux: bool = False):
        # Match the return format of StageAwareBackbone
        return self.backbone(x, need_aux)



def _looks_like_norm_name(name: str) -> bool:
    low = name.lower()

    return any(k in low for k in [
        ".norm", "bn", "groupnorm", "layernorm", "gn", "ln.", "ln_", "lnact", "ln_act", "lnproxy", "layernormproxy", "rmsnorm"
    ])



class HybridDCDATRT(nn.Module):
    """
    End‑to‑End Hybrid‑DCDAT‑RT detector (DAT + DCNv4 backbone + RT‑Deform Decoder)
    """
    def __init__(self,
                 num_classes:   int = 20,
                 num_queries:   int = 100,
                 depths:        List[int] = (2, 2, 2, 2),
                 drop_path_max: float = 0.0,
                 backbone_norm_factory: 'NormFactory' = None,
                 neck_norm_factory:     'NormFactory' = None,
                 use_layer_scale: bool = True,
                 layer_scale_init: float = 1.0,
                 # Decoder
                 d_model:    int = 320,
                 dec_layers: int = 4,
                 dn_queries: int = 100,
                 # Features
                 use_aux_loss: bool = True,
                 use_iou_aware: bool = False,
                 # LR multipliers
                 backbone_lr: float = 0.1,
                 head_lr:     float = 1.0,
                 # Dataset
                 voc_prior: bool = False,
                 # Decoder Encoder kontrolü
                 decoder_use_encoder: bool = True,
                 decoder_encoder_layers: int = 2,
                 decoder_encoder_drop_path_max: float = 0.1):
        super().__init__()

        # default NormFactory
        backbone_norm_factory = backbone_norm_factory or NormFactory("gn")
        neck_norm_factory     = neck_norm_factory     or NormFactory("gn")

        self.num_classes   = num_classes
        self.use_aux_loss  = use_aux_loss

        # ---------- Backbone (DAT + CSI + PGI) ----------
        self.backbone = StageAwareBackbone(
            depths          = depths,
            drop_path_max   = drop_path_max,
            num_classes     = num_classes,
            voc_prior       = voc_prior,
            norm_factory    = backbone_norm_factory,
            use_layer_scale = use_layer_scale,
            layer_scale_init= layer_scale_init
        )

        # ---------- Neck (Light‑BiFPN) ----------

        self.neck = LightBiFPN(
          in_channels=(128,256,640,768),
          out=d_model,
          repeats=2,
          use_spatial_fuse=True,
          fuse_groups=4,
          fuse_tau_init=1.4,
          fuse_learn_tau=True,
          fuse_eps=5e-4,
          grad_boost_low=0.1,     # << 0.5 → 0.3
          dp_top_second=0.01, dp_bot_second=0.01,
          no_blur_first=True,
          edge_enhance=True,
          edge_ls_init=0.03,
          norm_factory=neck_norm_factory,
      )

        # ---------- Decoder (RT‑Deform) ----------
        self.decoder = RTDeformDecoder(
            num_obj_classes = num_classes,
            include_background = True,
            num_queries     = num_queries,
            num_layers      = dec_layers,
            d_model         = d_model,
            dn_queries      = dn_queries,
            use_iou_aware   = use_iou_aware,
            use_encoder                 = decoder_use_encoder,
            encoder_layers              = decoder_encoder_layers,
            encoder_drop_path_max       = decoder_encoder_drop_path_max,
        )

        # (optional)
        self._lr_mult = {
            "backbone": backbone_lr,
            "head":     head_lr,
            "bias":     2.0,
            "dcn_bias": 10.0,
        }

    # ------------------------------------------------------------------ #
    def forward(self,
                x: torch.Tensor,
                targets: Optional[List[Dict]] = None) -> Dict[str, torch.Tensor]:
        """
        The output dictionary is compatible with RTDeformDecoder outputs:
          - ‘pred_logits’, ‘pred_boxes’, optional ‘pred_ious’
          - in training: ‘aux_outputs’, ‘dn_meta’, ‘query_selection_mask’
          - optional: ‘aux_dense’ (auxiliary dense headers from the backbone)
        """
        if not x.is_cuda:
            raise RuntimeError("HybridDCDATRT expects CUDA tensor input.")

        # Backbone
        if self.training and self.use_aux_loss:
            (p3, p4, p5, p6), aux_dense = self.backbone(x, need_aux=True)
        else:
            p3, p4, p5, p6 = self.backbone(x, need_aux=False)
            aux_dense      = None

        # Neck
        p3, p4, p5, p6 = self.neck(p3, p4, p5, p6)

        # Decoder
        if self.training and targets is not None:
            dec_out = self.decoder(p3, p4, p5, p6, targets)
        else:
            dec_out = self.decoder(p3, p4, p5, p6)

        if aux_dense is not None:
            dec_out["aux_dense"] = aux_dense
        return dec_out


    # ------------------------------------------------------------------ #

    def param_groups(self,
                    base_lr: float = 1e-4,
                    *,
                    weight_decay: float = 0.02,
                    bb_mult: float = 0.5,
                    dec_mult: float = 2.0,
                    bias_mult: float = 1.0,      # Bias boost
                    dcn_mult: float = 1.5,       # DCN offset/mask boost
                    rpb_mult: float = 5.0,       # RPB boost
                    gate_mult: float = 1.0,      # Gate params boost
                    ls_mult: float = 0.3,        # LayerScale reduction
                    pos_mult: float = 1.5):
        """
        parameter grouping with optimized LR scheduling.
        """
        import re

        buckets = {
            # Backbone
            "bb_w": [], "bb_b": [], "bb_norm": [],
            "dcn_off": [],  # DCN offset/mask

            # Neck / Head
            "hd_w": [], "hd_b": [], "hd_norm": [],
            "fuse_gate": [],  # Fusion gates

            # Decoder
            "dec_w": [], "dec_b": [], "dec_norm": [],
            "deform_off_b": [],  # Deformable attention bias

            # Special
            "pos_embed": [], "rpb_p": [], "ls_gamma": [],
            "dat_scale_p": [], "scalars_gain": [],
        }

        def is_norm_name(n: str) -> bool:
            return any(x in n for x in [".norm.", ".bn.", ".ln.", ".gn."])

        for name, param in self.named_parameters():
            if not param.requires_grad:
                continue

            # --- Special cases (prefix-independent) ---

            # RPB tables (needs high LR)
            if name.endswith(".rpb") or "relative_position_bias" in name:
                buckets["rpb_p"].append(param)
                continue

            # Positional embeddings
            if re.search(r"(level_embed(_seed)?|query_pos|query_feat|label_enc|pos_table)", name):
                buckets["pos_embed"].append(param)
                continue

            # LayerScale gamma
            if re.search(r"(layer_scales\.\d+\.weight|\.scale\.weight$|\.ls\.weight$)", name):
                buckets["ls_gamma"].append(param)
                continue

            # Scalar gates
            if any(name.endswith(x) for x in ["residual_scale", "_lambda_logit", "dc_res_logit", "seed_logit_obj", "seed_logit_grid"]):
                buckets["scalars_gain"].append(param)
                continue

            # DAT scale parameters
            if "dat_logit" in name or "dat_log_tau" in name:
                buckets["dat_scale_p"].append(param)
                continue

            # Neck fusion gates
            if "neck.fuse" in name and "log_tau" in name:
                buckets["fuse_gate"].append(param)
                continue

            # Deformable attention sampling bias
            if "sampling_offsets.bias" in name:
                buckets["deform_off_b"].append(param)
                continue

            # --- Module-based categorization ---

            # Backbone
            if name.startswith("backbone."):
                # DCN offset/mask (special treatment)
                if "offset_mask" in name and "dcn" in name:
                    buckets["dcn_off"].append(param)
                elif name.endswith(".bias") and not is_norm_name(name):
                    buckets["bb_b"].append(param)
                elif is_norm_name(name):
                    buckets["bb_norm"].append(param)
                else:
                    buckets["bb_w"].append(param)
                continue

            # Neck
            if name.startswith("neck."):
                if name.endswith(".bias") and not is_norm_name(name):
                    buckets["hd_b"].append(param)
                elif is_norm_name(name):
                    buckets["hd_norm"].append(param)
                else:
                    buckets["hd_w"].append(param)
                continue

            # Decoder (default)
            if name.endswith(".bias") and not is_norm_name(name):
                buckets["dec_b"].append(param)
            elif is_norm_name(name):
                buckets["dec_norm"].append(param)
            else:
                buckets["dec_w"].append(param)

        # --- Build parameter groups ---
        wd = weight_decay
        groups = []

        # Backbone groups
        if buckets["bb_w"]:
            groups.append({"params": buckets["bb_w"], "lr": base_lr*bb_mult, "weight_decay": wd, "name": "bb_w"})
        if buckets["bb_b"]:
            groups.append({"params": buckets["bb_b"], "lr": base_lr*bb_mult*bias_mult, "weight_decay": 0.0, "name": "bb_b"})
        if buckets["bb_norm"]:
            groups.append({"params": buckets["bb_norm"], "lr": base_lr*bb_mult, "weight_decay": 0.0, "name": "bb_norm"})

        # DCN offset/mask (HIGH LR!)
        if buckets["dcn_off"]:
            groups.append({"params": buckets["dcn_off"], "lr": base_lr*dcn_mult, "weight_decay": 0.0, "name": "dcn_off"})

        # Neck/Head groups
        if buckets["hd_w"]:
            groups.append({"params": buckets["hd_w"], "lr": base_lr, "weight_decay": wd, "name": "hd_w"})
        if buckets["hd_b"]:
            groups.append({"params": buckets["hd_b"], "lr": base_lr*bias_mult, "weight_decay": 0.0, "name": "hd_b"})
        if buckets["hd_norm"]:
            groups.append({"params": buckets["hd_norm"], "lr": base_lr, "weight_decay": 0.0, "name": "hd_norm"})

        # Decoder groups
        if buckets["dec_w"]:
            groups.append({"params": buckets["dec_w"], "lr": base_lr*dec_mult, "weight_decay": wd, "name": "dec_w"})
        if buckets["dec_b"]:
            groups.append({"params": buckets["dec_b"], "lr": base_lr*dec_mult*bias_mult, "weight_decay": 0.0, "name": "dec_b"})
        if buckets["dec_norm"]:
            groups.append({"params": buckets["dec_norm"], "lr": base_lr*dec_mult, "weight_decay": 0.0, "name": "dec_norm"})

        # Special parameters with custom LR
        if buckets["deform_off_b"]:
            groups.append({"params": buckets["deform_off_b"], "lr": base_lr*dcn_mult, "weight_decay": 0.0, "name": "deform_off_b"})
        if buckets["fuse_gate"]:
            groups.append({"params": buckets["fuse_gate"], "lr": base_lr*gate_mult, "weight_decay": 0.0, "name": "fuse_gate"})
        if buckets["rpb_p"]:
            groups.append({"params": buckets["rpb_p"], "lr": base_lr*rpb_mult, "weight_decay": 0.0, "name": "rpb_p"})
        if buckets["pos_embed"]:
            groups.append({"params": buckets["pos_embed"], "lr": base_lr*pos_mult, "weight_decay": 0.0, "name": "pos_embed"})
        if buckets["ls_gamma"]:
            groups.append({"params": buckets["ls_gamma"], "lr": base_lr*ls_mult, "weight_decay": 0.0, "name": "ls_gamma"})
        if buckets["dat_scale_p"]:
            groups.append({"params": buckets["dat_scale_p"], "lr": base_lr*gate_mult, "weight_decay": 0.0, "name": "dat_scale_p"})
        if buckets["scalars_gain"]:
            groups.append({"params": buckets["scalars_gain"], "lr": base_lr*gate_mult, "weight_decay": 0.0, "name": "scalars_gain"})

        # Validation
        in_groups = {id(p) for g in groups for p in g["params"]}
        missing = [n for n,p in self.named_parameters() if p.requires_grad and id(p) not in in_groups]
        assert not missing, f"Missing params in groups: {len(missing)} params (e.g., {missing[:5]})"

        # Debug logging (optional)
        if not hasattr(self, '_param_groups_logged'):
            print("\n" + "="*60)
            print("Parameter Groups Summary")
            print("="*60)
            for g in groups:
                n_params = sum(p.numel() for p in g["params"])
                if n_params > 0:
                    lr_mult = g["lr"] / base_lr
                    print(f"{g['name']:20s}: {n_params:10,} params | LR: {lr_mult:6.1f}x | WD: {g['weight_decay']:.3f}")
            print("="*60 + "\n")
            self._param_groups_logged = True

        return groups





# -----------------------------------------------------------------------------
# build_optimizer – returns only Optimizer (scheduler opsiyonel)
# -----------------------------------------------------------------------------
def build_optimizer(model: nn.Module,
                    base_lr: float = 2e-4,
                    weight_decay: float = 0.02,
                    *,
                    return_scheduler: bool = False):

    device = next(model.parameters()).device

    # ---------- 1) LAZY-PARAM WARM-UP (only backbone) ----------
    was_training = model.training
    model.train()  # PGI routes should be in train mode
    with torch.no_grad():
        for s in (640, 320):
            dummy = torch.zeros(1, 3, s, s, device=device)
            _ = model.backbone(dummy, need_aux=True)
    model.train(was_training)

    # ---------- 2) PARAM GRUPLARI ----------
    groups = model.param_groups(base_lr)

    # Norm weights WD=0 (GN/LNProxy vb.)
    ln_keys = ('.norm', '.ln_act', 'ln_proxy')
    #  Note: This check is heuristic by name; it can be customized as needed.
    for g in groups:
        params_in_group = set(map(id, g['params']))
        if any(any(k in n for k in ln_keys)
               for n, p in model.named_parameters() if id(p) in params_in_group):
            g["weight_decay"] = 0.0

    # ---------- 3) OPTIMIZER ----------
    optim = torch.optim.AdamW(
        groups, lr=base_lr,
        betas=(0.9, 0.999), eps=1e-6,
        weight_decay=weight_decay
    )

    if not return_scheduler:
        return optim

    sched = torch.optim.lr_scheduler.CosineAnnealingLR(
        optim, T_max=100, eta_min=base_lr * 0.05)
    return optim, sched


def build_model(num_classes: int = 20,
                norm: str = "gn",            # "gn", "lnp" veya "bn"
                use_layer_scale: bool = True,
                layer_scale_init: float = 1.0,
                **kwargs) -> HybridDCDATRT:
    """
    High-level constructor. The most commonly used settings come with defaults.
    HybridDCDATRT __init__ parameters (e.g., dec_layers, dn_queries,
    decoder_use_encoder, etc.) can be overridden via `kwargs`.
    """
    defaults = dict(
        # Backbone
        depths         = (2, 2, 4, 1),
        drop_path_max  = 0.2,
        # Decoder
        d_model        = 320,
        dec_layers     = 4,
        dn_queries     = 100,
        num_queries    = 200,
        use_aux_loss   = True,
        use_iou_aware  = False,
        # Dataset
        voc_prior      = False,

        decoder_use_encoder           = True,
        decoder_encoder_layers        = 2,
        decoder_encoder_drop_path_max = 0.1,
    )
    defaults.update(kwargs)

    nf = NormFactory(norm)
    model = HybridDCDATRT(
        num_classes           = num_classes,
        backbone_norm_factory = nf,
        neck_norm_factory     = nf,
        use_layer_scale       = use_layer_scale,
        layer_scale_init      = layer_scale_init,
        **defaults
    )
    # init_weights_improved(model)
    # apply_hybrid_fixup(model)
    # boost_relative_position_bias(model, std=0.05)
    return model
# -----------------------------------------------------------------------------
# Test dummy code
# -----------------------------------------------------------------------------

# if __name__ == "__main__":
#     import time

#     assert torch.cuda.is_available(), "CUDA required for this model."
#     device = torch.device("cuda")

#     print("="*80)
#     print("Building Hybrid-DCDAT-RT Model...")
#     print("="*80)

#     model = build_model(
#         num_classes=80,
#         depths=[2, 2, 2, 2],  # ResNet-50 like depth
#         drop_path_max=0,
#         num_queries=200,
#         voc_prior=False,norm="gn"  # Use COCO prior
#     ).to(device)
#     optimizer = build_optimizer(model, base_lr=2e-4)
#     # Test forward pass
#     print("\n🚀 Testing forward pass...")
#     model.eval()
#     dummy = torch.randn(2, 3, 640, 640, device=device)

#     with torch.no_grad():
#         out = model(dummy)
#         print("✅ Forward pass successful!")
#         print(f"Output keys: {list(out.keys())}")
#         print(f"Predictions shape: {out['pred_logits'].shape}, {out['pred_boxes'].shape}")

#     # Test training mode with targets
#     print("\n🚀 Testing training mode...")
#     model.train()

#     # Create more realistic targets with proper box format
#     targets = []
#     for _ in range(2):  # batch size 2
#         num_objs = torch.randint(1, 10, (1,)).item()
#         # Generate boxes in (cx, cy, w, h) format, all normalized to [0, 1]
#         centers = torch.rand(num_objs, 2, device=device)
#         sizes = torch.rand(num_objs, 2, device=device) * 0.5  # max size 0.5
#         boxes = torch.cat([centers, sizes], dim=1)

#         # Ensure boxes are valid (centers must be at least half-size from borders)
#         half_sizes = boxes[:, 2:4] / 2
#         boxes[:, 0] = boxes[:, 0].clamp(min=half_sizes[:, 0], max=1-half_sizes[:, 0])
#         boxes[:, 1] = boxes[:, 1].clamp(min=half_sizes[:, 1], max=1-half_sizes[:, 1])

#         targets.append({
#             'boxes': boxes,
#             'labels': torch.randint(0, 80, (num_objs,), device=device)
#         })

#     out = model(dummy, targets)
#     print("✅ Training mode successful!")
#     print(f"Output keys: {list(out.keys())}")
#     if 'aux_dense' in out:
#         print(f"Auxiliary dense outputs: {list(out['aux_dense'].keys())}")
#     if 'dn_meta' in out:
#         print(f"Denoising queries: {out['dn_meta']['dn_queries']}")

#     # Test gradient flow
#     print("\n🚀 Testing gradient flow...")
#     loss = out['pred_logits'].sum() + out['pred_boxes'].sum()
#     loss.backward()

#     # Check that gradients flow through all parts
#     has_grad = {
#         'backbone': any(p.grad is not None for n, p in model.named_parameters() if 'backbone' in n),
#         'neck': any(p.grad is not None for n, p in model.named_parameters() if 'neck' in n),
#         'decoder': any(p.grad is not None for n, p in model.named_parameters() if 'decoder' in n),
#     }

#     for module, has in has_grad.items():
#         print(f"  {module}: {'✅' if has else '❌'} gradients")

#     # Test parameter groups
#     print("\n🚀 Testing parameter groups...")
#     param_groups = model.param_groups(base_lr=2e-4)
#     print(f"Number of parameter groups: {len(param_groups)}")
#     for i, group in enumerate(param_groups):
#         print(f"  Group {i}: {len(group['params'])} params, lr={group['lr']:.6f}, wd={group['weight_decay']}")

#     print("\n" + "="*80)
#     print("✅ All tests passed! Model is ready.")
#     print("="*80)

In [ ]:
#mini imagenet 100 for pretrain

from datasets import load_dataset
train_ds = load_dataset("timm/mini-imagenet", split="train")        # 50 000 train
val_ds   = load_dataset("timm/mini-imagenet", split="validation")   # 10 000 val
print(train_ds)

In [ ]:
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torch
import torchvision.transforms as T
from torchvision.transforms import (
    Compose, RandomResizedCrop, RandomHorizontalFlip,
    RandAugment, ColorJitter, RandomGrayscale, RandomErasing,
    ToTensor, Normalize
)

class MiniImageNetDataset(Dataset):
    def __init__(self, hf_dataset, transform=None):
        self.dataset = hf_dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample = self.dataset[idx]
        image = sample['image']
        label = sample['label']

        # Convert the image to PIL format
        if isinstance(image, list):
            image = image[0] if len(image) > 0 else None

        if isinstance(image, np.ndarray):
            image = Image.fromarray(image)

        # Always  RGB
        if image.mode != 'RGB':
            image = image.convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label



train_tf = Compose([
    # ❶ Geometric transformations
    RandomResizedCrop(
        224,
        scale=(0.08, 1.0),
        ratio=(3/4, 4/3),
        antialias=True
    ),
    RandomHorizontalFlip(p=0.5),

    # ❷ RandAugment  (2 op, magnitude=9)  → random blur, solarize, rotate…
    RandAugment(num_ops=2, magnitude=9),

    # ❸ Additional color jitter (slight variation in addition to that within RA)
    ColorJitter(0.4, 0.4, 0.4, 0.1),

    # ❹ Tensöre & normalize
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406],
              std=[0.229, 0.224, 0.225]),

    # ❺ RandomErasing
    RandomErasing(
        p=0.2,
        scale=(0.02, 0.2),
        ratio=(0.3, 3.3),
        value='random',   # rastgele RGB
        inplace=False
    ),
])

# ---------------- VAL PIPELINE ------------------
val_tf = Compose([
    T.Resize(256, antialias=True),
    T.CenterCrop(224),
    T.Lambda(lambda x: x.convert("RGB") if x.mode != "RGB" else x),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406],
              std=[0.229, 0.224, 0.225]),
])
# DataLoaders
train_dataset = MiniImageNetDataset(train_ds, transform=train_tf)
val_dataset = MiniImageNetDataset(val_ds, transform=val_tf)

train_dl = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    drop_last=True
)

val_dl = DataLoader(
    val_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

# Test
print("DataLoader test...")
for images, labels in train_dl:
    print(f"✓ Batch shape: {images.shape}, Labels: {labels.shape}")
    print(f"✓ Image min: {images.min():.2f}, max: {images.max():.2f}")
    break

print(f"\n✓ DataLoaders are ready!")
print(f"  Train batches: {len(train_dl)}")
print(f"  Val batches: {len(val_dl)}")

In [ ]:
# =====================================================
# Production-ready training script with safe resume & detection export
# =====================================================
import os, math, json, warnings
from copy import deepcopy
from datetime import datetime
from typing import Tuple, Dict

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import LambdaLR
from tqdm import tqdm
import numpy as np


# Colab Drive (optional)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Mixup
from timm.data import Mixup
from timm.loss import SoftTargetCrossEntropy

# ---- Numeric safety: FP32 strict (no TF32) ----
torch.set_default_dtype(torch.float32)
torch.backends.cuda.matmul.allow_tf32 = False
torch.backends.cudnn.allow_tf32 = False


# =====================================================
# 1) CONFIG
# =====================================================
MODEL_CONFIG = {
    'num_classes': 100,
    'include_neck': True,
    'multi_head': True,
    'depths': (2, 2, 4, 1),
    'drop_path_max': 0.15,
    'norm': 'gn',
    'use_layer_scale': True,
    'layer_scale_init': 0.3,
    'd_model': 320,
    'backbone_channels': (128, 256, 640, 768),
}

TRAINING_CONFIG = {
    'dataset': 'Mini-ImageNet',
    'batch_size': 128,
    'num_epochs': 200,
    'base_lr': 2e-4,
    'weight_decay':0.02,
    'warmup_epochs': 1,
    'accumulation_steps': 1,
    'use_ema': True,
    'ema_decay': 0.999,
    'precision': 'fp32_strict',
}

# =====================================================
# 2) SMALL UTILS
# =====================================================
def set_seed(seed=42):
    import random
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)

def clip_dcnv4_grads(model: nn.Module, max_norm: float = 0.12):
    """Clip only DCN/Deformable sampling offsets (if present). Safe no-op otherwise."""
    sub = [p for n, p in model.named_parameters()
           if p.grad is not None and ('offset_mask' in n or 'sampling_offsets' in n)]
    if sub:
        torch.nn.utils.clip_grad_norm_(sub, max_norm)

def get_aux_weights(epoch, total_epochs):
    prog = epoch / total_epochs
    if epoch >= 190:
        return {'main': 1.0, 'p3': 0.05, 'p4': 0.1, 'p5': 0.1, 'p6': 0.05}
    elif epoch >= 180:
        return {'main': 1.0, 'p3': 0.05, 'p4': 0.1, 'p5': 0.1, 'p6': 0.05}
    elif prog < 0.3:
        return {'main': 1.0, 'p3': 0.3, 'p4': 0.5, 'p5': 0.5, 'p6': 0.3}
    elif prog < 0.7:
        return {'main': 1.0, 'p3': 0.2, 'p4': 0.4, 'p5': 0.4, 'p6': 0.2}
    else:
        return {'main': 1.0, 'p3': 0.1, 'p4': 0.2, 'p5': 0.2, 'p6': 0.1}

def adjust_mixup_params(epoch):
    if epoch < 10:   return 0.0, 0.0, 0.0
    if epoch < 30:   return 0.3, 0.3, 0.3
    if epoch < 100:  return 0.9, 0.5, 0.9
    if epoch < 160:  return 0.9, 0.6, 1.0
    return 0.9, 0.6, 1.1

def compute_auxiliary_loss(outputs, targets, criterion, weights):
    losses = {}
    for key in ['main', 'p3', 'p4', 'p5', 'p6']:
        if key in outputs:
            losses[key] = criterion(outputs[key], targets) * weights[key]
    total_loss = sum(losses.values())
    return total_loss, losses

def safe_load(module: nn.Module, state_dict: Dict[str, torch.Tensor], strict: bool, name: str):
    """Use strict mode; if an error occurs, try to strict=False and report it."""
    try:
        msg = module.load_state_dict(state_dict, strict=strict)
        if not strict:
            print(f"[{name}] load_state_dict(strict=False) → missing={len(msg.missing_keys)}, unexpected={len(msg.unexpected_keys)}")
            if msg.missing_keys:   print(f"  missing:   {msg.missing_keys[:10]}{' ...' if len(msg.missing_keys)>10 else ''}")
            if msg.unexpected_keys:print(f"  unexpected:{msg.unexpected_keys[:10]}{' ...' if len(msg.unexpected_keys)>10 else ''}")
    except RuntimeError as e:
        print(f"[{name}] strict=True failed: {e}\n→ trying strict=False ")
        msg = module.load_state_dict(state_dict, strict=False)
        print(f"[{name}] load_state_dict(strict=False) → missing={len(msg.missing_keys)}, unexpected={len(msg.unexpected_keys)}")
    return module

def filter_backbone_for_detection(backbone_sd: dict) -> dict:
    """Exclude class-dependent layers within the backbone (aux heads, etc.)"""
    drop_substrings = ('aux_head_', 'cls_head', 'aux_cls_heads', 'aux_box_heads')
    return {k: v for k, v in backbone_sd.items()
            if not any(sub in k for sub in drop_substrings)}

def prime_backbone_for_optimizer(model: nn.Module):
    """A quick dry-run to ensure the initialization gate/adapter paths."""
    dev = next(model.parameters()).device
    was_training = model.training
    model.train()
    with torch.no_grad():
        for s in (640, 320):
            dummy = torch.zeros(1, 3, s, s, device=dev)
            _ = model.backbone(dummy, need_aux=True)
    model.train(was_training)

# =====================================================
# 3) EMA (optional)
# =====================================================
class EMAModel:
    def __init__(self, model, decay=0.9995, device='cuda'):
        self.decay = decay
        self.ema = deepcopy(model).to(device)
        self.ema.eval()
        for p in self.ema.parameters():
            p.requires_grad = False
        print(f"✅ EMA initialized (decay={decay})")

    @torch.no_grad()
    def update(self, model):
        src = model.module if hasattr(model, 'module') else model
        sdict = src.state_dict()
        edict = self.ema.state_dict()
        for name, e in edict.items():
            if name not in sdict:
                continue
            x = sdict[name]
            if e.shape != x.shape:
                continue
            if x.dtype in (torch.float32, torch.float16, torch.bfloat16):
                if ('running_mean' in name or 'running_var' in name or
                    'num_batches_tracked' in name):
                    e.copy_(x)
                else:
                    e.mul_(self.decay).add_(x, alpha=1 - self.decay)
            else:
                e.copy_(x)

    def state_dict(self):
        return self.ema.state_dict()

    def load_state_dict(self, sd):
        self.ema.load_state_dict(sd, strict=False)

    @property
    def model(self):
        return self.ema

# =====================================================
# 4) MODEL(for pretrain)
# =====================================================
class DATClassifierMSP(nn.Module):
    """Multi-scale, optional neck, multi-head classifier."""
    def __init__(self,
                 num_classes: int = 100,
                 include_neck: bool = True,
                 multi_head: bool = True,
                 depths: Tuple[int,int,int,int] = (2,2,4,1),
                 drop_path_max: float = 0.15,
                 norm: str = "gn",
                 use_layer_scale: bool = True,
                 layer_scale_init: float = 0.3,
                 d_model: int = 320,
                 backbone_channels: Tuple[int,int,int,int] = (128, 256, 640, 768)):
        super().__init__()
        self.config = dict(
            num_classes=num_classes, include_neck=include_neck, multi_head=multi_head,
            depths=depths, drop_path_max=drop_path_max, norm=norm,
            use_layer_scale=use_layer_scale, layer_scale_init=layer_scale_init,
            d_model=d_model, backbone_channels=backbone_channels
        )
        self.multi_head = multi_head

        nf = NormFactory(norm)



        self.backbone = StageAwareBackbone(
            depths          = depths,
            drop_path_max   = drop_path_max,
            num_classes     = num_classes,
            voc_prior       = False,
            norm_factory    = NormFactory("gn"),
            use_layer_scale = use_layer_scale,
            layer_scale_init= layer_scale_init
        )

        # ---------- Neck (Light‑BiFPN) (optional neck) ----------


        self.neck = None # optional neck
        self.in_dims = backbone_channels
        if include_neck:
            self.neck = LightBiFPN(
          in_channels=(128,256,640,768),
          out=d_model,
          repeats=2,
          use_spatial_fuse=True,
          fuse_groups=4,
          fuse_tau_init=1.4,
          fuse_learn_tau=True,
          fuse_eps=5e-4,
          grad_boost_low=0.12,
          dp_top_second=0.01, dp_bot_second=0.01,
          no_blur_first=False,
          edge_enhance=True,
          edge_ls_init=0.03,
          norm_factory=NormFactory("gn"),)
            self.in_dims = (d_model,)*4

        if multi_head:
            def _make_head(c, ncls):
                head = nn.Sequential(
                    nn.AdaptiveAvgPool2d(1),
                    nn.Flatten(),
                    nn.Linear(c, ncls),
                )
                nn.init.trunc_normal_(head[-1].weight, std=0.02)
                nn.init.zeros_(head[-1].bias)
                return head
            self.head_p3 = _make_head(self.in_dims[0], num_classes)
            self.head_p4 = _make_head(self.in_dims[1], num_classes)
            self.head_p5 = _make_head(self.in_dims[2], num_classes)
            self.head_p6 = _make_head(self.in_dims[3], num_classes)
        else:
            self.cls_head = nn.Linear(sum(self.in_dims), num_classes)
            nn.init.zeros_(self.cls_head.bias)

    def forward(self, x):
        x = x.float()
        p3, p4, p5, p6 = self.backbone(x, need_aux=False)
        if self.neck is not None:
            p3, p4, p5, p6 = self.neck(p3, p4, p5, p6)
        if self.multi_head:
            out_p3 = self.head_p3(p3)
            out_p4 = self.head_p4(p4)
            out_p5 = self.head_p5(p5)
            out_p6 = self.head_p6(p6)
            if self.training:
                return {'p3': out_p3, 'p4': out_p4, 'p5': out_p5, 'p6': out_p6, 'main': (out_p5+out_p6)/2}
            return (out_p5 + out_p6) / 2
        feats = torch.cat([f.mean(dim=[-2, -1]) for f in (p3, p4, p5, p6)], 1)
        return self.cls_head(feats)

    def param_groups(self,
                     base_lr: float = 2e-4,
                     *,
                     weight_decay: float = 0.02,
                     bb_mult: float = 1.0,
                     dec_mult: float = 1.50,
                     bias_mult: float = 1.0,
                     dcn_mult: float = 2.0,
                     rpb_mult: float = 8.0,
                     gate_mult: float = 1.0,
                     ls_mult: float = 0.3,
                     pos_mult: float = 1.0):
        import re
        buckets = {k: [] for k in [
            "bb_w","bb_b","bb_norm","dcn_off",
            "hd_w","hd_b","hd_norm","fuse_gate",
            "dec_w","dec_b","dec_norm","deform_off_b",
            "pos_embed","rpb_p","ls_gamma","dat_scale_p","scalars_gain"
        ]}
        def is_norm_name(n: str) -> bool:
            return any(x in n for x in [".norm.", ".bn.", ".ln.", ".gn."])

        for name, p in self.named_parameters():
            if not p.requires_grad: continue
            # specials (prefix independent)
            if name.endswith(".rpb") or "relative_position_bias" in name:
                buckets["rpb_p"].append(p); continue
            if re.search(r"(level_embed(_seed)?|query_pos|query_feat|label_enc|pos_table)", name):
                buckets["pos_embed"].append(p); continue
            if re.search(r"(layer_scales\.\d+\.weight|\.scale\.weight$|\.ls\.weight$)", name):
                buckets["ls_gamma"].append(p); continue
            if any(name.endswith(x) for x in ["residual_scale","_lambda_logit","dc_res_logit","seed_logit_obj","seed_logit_grid"]):
                buckets["scalars_gain"].append(p); continue
            if "dat_logit" in name or "dat_log_tau" in name:
                buckets["dat_scale_p"].append(p); continue
            if "neck.fuse" in name and "log_tau" in name:
                buckets["fuse_gate"].append(p); continue
            if "sampling_offsets.bias" in name:
                buckets["deform_off_b"].append(p); continue

            # module-based
            if name.startswith("backbone."):
                if "offset_mask" in name and "dcn" in name:
                    buckets["dcn_off"].append(p)
                elif name.endswith(".bias") and not is_norm_name(name):
                    buckets["bb_b"].append(p)
                elif is_norm_name(name):
                    buckets["bb_norm"].append(p)
                else:
                    buckets["bb_w"].append(p)
                continue

            if name.startswith("neck."):
                if name.endswith(".bias") and not is_norm_name(name):
                    buckets["hd_b"].append(p)
                elif is_norm_name(name):
                    buckets["hd_norm"].append(p)
                else:
                    buckets["hd_w"].append(p)
                continue

            # default = "decoder-ish"
            if name.endswith(".bias") and not is_norm_name(name):
                buckets["dec_b"].append(p)
            elif is_norm_name(name):
                buckets["dec_norm"].append(p)
            else:
                buckets["dec_w"].append(p)

        groups = []
        wd = weight_decay
        # backbone
        if buckets["bb_w"]:   groups.append({"params": buckets["bb_w"],   "lr": base_lr*bb_mult,           "weight_decay": wd,  "name": "bb_w"})
        if buckets["bb_b"]:   groups.append({"params": buckets["bb_b"],   "lr": base_lr*bb_mult*bias_mult, "weight_decay": 0.0, "name": "bb_b"})
        if buckets["bb_norm"]:groups.append({"params": buckets["bb_norm"],"lr": base_lr*bb_mult,           "weight_decay": 0.0, "name": "bb_norm"})
        if buckets["dcn_off"]:groups.append({"params": buckets["dcn_off"],"lr": base_lr*dcn_mult,          "weight_decay": 0.0, "name": "dcn_off"})
        # neck
        if buckets["hd_w"]:   groups.append({"params": buckets["hd_w"],   "lr": base_lr,                   "weight_decay": wd,  "name": "hd_w"})
        if buckets["hd_b"]:   groups.append({"params": buckets["hd_b"],   "lr": base_lr*bias_mult,         "weight_decay": 0.0, "name": "hd_b"})
        if buckets["hd_norm"]:groups.append({"params": buckets["hd_norm"],"lr": base_lr,                   "weight_decay": 0.0, "name": "hd_norm"})
        # decoder-ish
        if buckets["dec_w"]:  groups.append({"params": buckets["dec_w"],  "lr": base_lr*dec_mult,          "weight_decay": wd,  "name": "dec_w"})
        if buckets["dec_b"]:  groups.append({"params": buckets["dec_b"],  "lr": base_lr*dec_mult*bias_mult,"weight_decay": 0.0, "name": "dec_b"})
        if buckets["dec_norm"]:groups.append({"params": buckets["dec_norm"],"lr": base_lr*dec_mult,        "weight_decay": 0.0, "name": "dec_norm"})
        # specials
        if buckets["deform_off_b"]: groups.append({"params": buckets["deform_off_b"], "lr": base_lr*dcn_mult, "weight_decay": 0.0, "name": "deform_off_b"})
        if buckets["fuse_gate"]:    groups.append({"params": buckets["fuse_gate"],    "lr": base_lr*gate_mult,"weight_decay": 0.0, "name": "fuse_gate"})
        if buckets["rpb_p"]:        groups.append({"params": buckets["rpb_p"],        "lr": base_lr*rpb_mult, "weight_decay": 0.0, "name": "rpb_p"})
        if buckets["pos_embed"]:    groups.append({"params": buckets["pos_embed"],    "lr": base_lr*pos_mult, "weight_decay": 0.0, "name": "pos_embed"})
        if buckets["ls_gamma"]:     groups.append({"params": buckets["ls_gamma"],     "lr": base_lr*ls_mult,  "weight_decay": 0.0, "name": "ls_gamma"})
        if buckets["dat_scale_p"]:  groups.append({"params": buckets["dat_scale_p"],  "lr": base_lr*gate_mult,"weight_decay": 0.0, "name": "dat_scale_p"})
        if buckets["scalars_gain"]: groups.append({"params": buckets["scalars_gain"], "lr": base_lr*gate_mult,"weight_decay": 0.0, "name": "scalars_gain"})

        # validation
        in_groups = {id(p) for g in groups for p in g["params"]}
        missing = [n for n,p in self.named_parameters() if p.requires_grad and id(p) not in in_groups]
        assert not missing, f"Missing params in groups: {len(missing)} params (e.g., {missing[:5]})"

        if not hasattr(self, '_param_groups_logged'):
            print("\n" + "="*60)
            print("Parameter Groups Summary")
            print("="*60)
            for g in groups:
                n_params = sum(p.numel() for p in g["params"])
                if n_params > 0:
                    lr_mult = g["lr"] / base_lr
                    print(f"{g['name']:20s}: {n_params:10,} params | LR: {lr_mult:6.1f}x | WD: {g['weight_decay']:.3f}")
            print("="*60 + "\n")
            self._param_groups_logged = True
        return groups

# =====================================================
# 5) BUILD / RESUME
# =====================================================
def create_model_from_config(config: dict, device='cuda'):
    model = DATClassifierMSP(**config).to(device)
    model = model.float()
    for p in model.parameters():
        p.data = p.data.float()
    init_weights_improved(model)
    apply_hybrid_fixup(model)
    boost_relative_position_bias(model, std=0.05)
    return model

def build_optimizer(model: nn.Module, base_lr: float, weight_decay: float, warmup_epochs: int, total_epochs: int):
    # ensure gates/adapters are “primed”
    prime_backbone_for_optimizer(model)

    params = model.param_groups(base_lr)
    # heuristic WD=0 for norms (in case any slipped through)
    ln_keys = ('.norm', '.ln', '.bn', '.gn')
    name_by_id = {id(p): n for n, p in model.named_parameters()}
    for g in params:
        if any(any(k in name_by_id.get(id(p), '') for k in ln_keys) for p in g['params']):
            g['weight_decay'] = 0.0

    optim = torch.optim.AdamW(params, lr=base_lr, betas=(0.9, 0.999), eps=1e-6, weight_decay=weight_decay)

    def lr_lambda(cur_epoch):
        if cur_epoch < warmup_epochs:
            return (cur_epoch + 1) / warmup_epochs
        t = (cur_epoch - warmup_epochs) / max(1, total_epochs - warmup_epochs)
        return 0.5 * (1 + math.cos(math.pi * t))

    sched = LambdaLR(optim, lr_lambda)
    return optim, sched

def save_pretrained_weights_for_detection(checkpoint_dir, checkpoint_data, model_config, suffix=""):
    print(f"\n💾 Saving Pretrained backbone + (neck optional) {suffix}...")
    ema_bb = checkpoint_data.get('ema_backbone_state_dict')
    raw_bb = checkpoint_data.get('backbone_state_dict')
    bb_sd = ema_bb if ema_bb is not None else raw_bb
    nk_sd = checkpoint_data.get('ema_neck_state_dict') or checkpoint_data.get('neck_state_dict')

    if bb_sd is None:
        warnings.warn("No backbone_state_dict in checkpoint; skipping export.")
        return

    bb_filtered = filter_backbone_for_detection(bb_sd)

    detection_info = {
        'backbone_state_dict': bb_filtered,
        'neck_state_dict': nk_sd,
        'accuracy': float(checkpoint_data.get('best_val_acc', 0.0)),
        'model_type': 'backbone_and_neck' if nk_sd is not None else 'backbone_only',
        'is_ema': ema_bb is not None,
        'architecture_config': {
            'depths': model_config['depths'],
            'drop_path_max': model_config['drop_path_max'],
            'norm': model_config['norm'],
            'use_layer_scale': model_config['use_layer_scale'],
            'layer_scale_init': model_config['layer_scale_init'],
            'backbone_channels': model_config['backbone_channels'],
            'd_model': model_config['d_model'],
            'include_neck': model_config['include_neck'],
            'multi_head': model_config.get('multi_head', False),
        },
        'training_config': checkpoint_data.get('training_config', {}),
        'info': 'Mini-ImageNet pretrained (EMA weights preferred, aux-heads stripped)',
    }
    out_path = os.path.join(checkpoint_dir, 'pretrained_backbone_neck.pth')
    torch.save(detection_info, out_path)
    print(f"✅ Saved → {out_path}")

# =====================================================
# 6) TRAIN / EVAL
# =====================================================
def train_epoch(model, dataloader, criterion, mixup_criterion, mixup_fn, optimizer,
                epoch, num_epochs, warmup_epochs, base_lr, accumulation_steps=2,
                ema_model: EMAModel=None, device='cuda'):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    aux_weights = get_aux_weights(epoch, num_epochs)

    # mixup schedule
    mix_prob, mix_alpha, cut_alpha = adjust_mixup_params(epoch)
    mixup_fn.mix_prob = mix_prob
    mixup_fn.mixup_alpha = mix_alpha
    mixup_fn.cutmix_alpha = cut_alpha
    use_mixup = mix_prob > 0 and epoch >= 5

    pbar = tqdm(dataloader, desc=f'Epoch {epoch+1} Training')
    optimizer.zero_grad(set_to_none=True)

    for batch_idx, (inputs, targets) in enumerate(pbar):
        inputs = inputs.to(device).float()
        targets_original = targets.to(device)

        if use_mixup:
            inputs, targets_mixed = mixup_fn(inputs, targets_original)
            outputs = model(inputs)
            if isinstance(outputs, dict):
                loss = mixup_criterion(outputs['main'], targets_mixed)
                for k in ['p3','p4','p5','p6']:
                    if k in outputs and k in aux_weights:
                        loss += aux_weights[k] * mixup_criterion(outputs[k], targets_mixed)
                loss = loss / accumulation_steps
                with torch.no_grad():
                    _, pred = outputs['main'].max(1)
                    correct += pred.eq(targets_original).sum().item()
            else:
                loss = mixup_criterion(outputs, targets_mixed) / accumulation_steps
                with torch.no_grad():
                    _, pred = outputs.max(1)
                    correct += pred.eq(targets_original).sum().item()
        else:
            outputs = model(inputs)
            if isinstance(outputs, dict):
                loss, _ = compute_auxiliary_loss(outputs, targets_original, criterion, aux_weights)
                loss = loss / accumulation_steps
                _, pred = outputs['main'].max(1)
            else:
                loss = criterion(outputs, targets_original) / accumulation_steps
                _, pred = outputs.max(1)
            correct += pred.eq(targets_original).sum().item()

        loss.backward()

        if (batch_idx + 1) % accumulation_steps == 0 or (batch_idx + 1) == len(dataloader):
            clip_dcnv4_grads(model, max_norm=0.12)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)
            if ema_model is not None:
                ema_model.update(model)

        running_loss += loss.item() * accumulation_steps * inputs.size(0)
        total += inputs.size(0)

        current_lr = optimizer.param_groups[0]['lr']
        acc_display = f'{100.*correct/total:.2f}%' + ('*' if use_mixup else '')
        pbar.set_postfix(Loss=f'{running_loss/total:.4f}', Acc=acc_display,
                         LR=f'{current_lr:.6f}', Mix= f'α={mix_alpha:.1f}' if use_mixup else 'OFF')

    dataset_size = len(dataloader.dataset)
    return running_loss / dataset_size, 100. * correct / total

@torch.no_grad()
def validate(model, dataloader, criterion, ema_model: EMAModel=None, device='cuda'):
    eval_model = ema_model.model if ema_model else model
    eval_model.eval()
    running_loss, correct, total = 0.0, 0, 0
    pbar = tqdm(dataloader, desc='Validation' + (' (EMA)' if ema_model else ''))
    for inputs, targets in pbar:
        inputs = inputs.to(device).float()
        targets = targets.to(device)
        outputs = eval_model(inputs)
        if isinstance(outputs, dict):
            loss = criterion(outputs['main'], targets)
            _, pred = outputs['main'].max(1)
        else:
            loss = criterion(outputs, targets)
            _, pred = outputs.max(1)
        running_loss += loss.item() * targets.size(0)
        total += targets.size(0)
        correct += pred.eq(targets).sum().item()
        pbar.set_postfix(Loss=f'{running_loss/total:.4f}', Acc=f'{100.*correct/total:.2f}%')
    dataset_size = len(dataloader.dataset)
    return running_loss / dataset_size, 100. * correct / total

def save_checkpoint(state, filename):
    torch.save(state, filename)

# =====================================================
# 7) MAIN
# =====================================================
set_seed(123)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# ---- Resume config ----
RESUME_PATH = "/content/ablation4.pth"   # If there is a file, it will be read automatically.
LOAD_WEIGHTS_ONLY = False              # True → only weights, optim/sched/epoch are reset

# ---- Decide which config to build with (for full resume, prefer ckpt config) ----
ckpt_meta = None
if RESUME_PATH and os.path.exists(RESUME_PATH):
    ckpt_meta = torch.load(RESUME_PATH, map_location='cpu')
    ckpt_cfg = ckpt_meta.get('model_config', MODEL_CONFIG)
    if not LOAD_WEIGHTS_ONLY:
        print("ℹ️ Full resume: model configuration is retrieved from the checkpoint.")
        model_to_build_cfg = ckpt_cfg
    else:
        print("ℹ️ Weights-only load: will be built with the current MODEL_CONFIG.")
        model_to_build_cfg = MODEL_CONFIG
else:
    model_to_build_cfg = MODEL_CONFIG

# ---- Build model ----
model = create_model_from_config(model_to_build_cfg, device)
print(f"✓ Model is ready")
print("  Config:")
for k, v in model_to_build_cfg.items(): print(f"    {k}: {v}")
print("  Params (M):")
print(f"    - Backbone: {sum(p.numel() for n,p in model.named_parameters() if n.startswith('backbone'))/1e6:.2f}")
print(f"    - Neck:     {sum(p.numel() for n,p in model.named_parameters() if n.startswith('neck'))/1e6:.2f}")
print(f"    - Heads:    {sum(p.numel() for n,p in model.named_parameters() if n.startswith('head_'))/1e6:.2f}")
print(f"    - Total:    {sum(p.numel() for p in model.parameters())/1e6:.2f}")



# ---- Training setup ----
num_epochs       = TRAINING_CONFIG['num_epochs']
base_lr          = TRAINING_CONFIG['base_lr']
weight_decay     = TRAINING_CONFIG['weight_decay']
warmup_epochs    = TRAINING_CONFIG['warmup_epochs']
accum_steps      = TRAINING_CONFIG['accumulation_steps']
use_ema          = TRAINING_CONFIG['use_ema']
ema_decay        = TRAINING_CONFIG['ema_decay']

criterion        = nn.CrossEntropyLoss(label_smoothing=0.1)
mixup_criterion  = SoftTargetCrossEntropy()
mixup_fn = Mixup(mixup_alpha=0.0, cutmix_alpha=0.0, cutmix_minmax=None,
                 prob=0.0, switch_prob=0.5, mode='batch',
                 label_smoothing=0.1, num_classes=model.config['num_classes'])

optimizer, scheduler = build_optimizer(model, base_lr, weight_decay, warmup_epochs, num_epochs)

# ---- Checkpoint dir ----
BASE_DIR = '/content/drive/MyDrive'
EXPERIMENT_NAME = 'mini_imagenet_neck_multihead_final'
checkpoint_dir = f'{BASE_DIR}/{EXPERIMENT_NAME}_{datetime.now().strftime("%Y%m%d_%H%M%S")}'
os.makedirs(checkpoint_dir, exist_ok=True)

with open(os.path.join(checkpoint_dir, 'config.json'), 'w') as f:
    json.dump({'model_config': model_to_build_cfg, 'training_config': TRAINING_CONFIG,
               'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S")}, f, indent=4)

start_epoch = 0
best_val_acc = 0.0
training_history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

# ---- EMA ----
ema_model = EMAModel(model, decay=ema_decay, device=device) if use_ema else None

# ---- Resume logic ----
if ckpt_meta is not None:
    print(f"\n📁 Checkpoint is loading: {RESUME_PATH}")
    if LOAD_WEIGHTS_ONLY:
        # weights only
        safe_load(model, ckpt_meta['model_state_dict'], strict=False, name='model')
        if ema_model and ckpt_meta.get('ema_state_dict') is not None:
            ema_model.load_state_dict(ckpt_meta['ema_state_dict'])
            print("✅ EMA weights are loaded")
        start_epoch = 0
        best_val_acc = 0.0
        training_history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
        print("⚠️ Weights-only: epoch/optim/sched reset.")
    else:
        # full resume (prefer strict)
        safe_load(model, ckpt_meta['model_state_dict'], strict=True, name='model')
        if ema_model and ckpt_meta.get('ema_state_dict') is not None:
            ema_model.load_state_dict(ckpt_meta['ema_state_dict'])
            print("✅ EMA loaded")
        try:
            optimizer.load_state_dict(ckpt_meta['optimizer_state_dict'])
            print("✅ Optimizer loaded")
        except Exception as e:
            print(f"⚠️ Optimizer load failed (fresh started): {e}")
        try:
            scheduler.load_state_dict(ckpt_meta['scheduler_state_dict'])
            scheduler.last_epoch = ckpt_meta.get('epoch', 0) - 1
            print("✅ Scheduler loaded")
        except Exception as e:
            print(f"⚠️ Scheduler load failed (fresh): {e}")
        start_epoch   = ckpt_meta.get('epoch', 0)
        best_val_acc  = ckpt_meta.get('best_val_acc', 0.0)
        training_history = ckpt_meta.get('history', training_history)
        print(f"\n📊 Resume summary → start_epoch={start_epoch} | best={best_val_acc:.2f}%")

# =====================================================
# 8) TRAIN LOOP
# =====================================================
print("\n" + "="*70)
print("TRAINING is Starting...")
print("="*70)
print(f"⚡ TF32: OFF")
print(f"⚡ Drop Path: {model.config['drop_path_max']}")
print(f"⚡ Base LR : {base_lr}")
print(f"⚡ Multi-head: {'ON' if model.multi_head else 'OFF'}")

last_save_epoch = start_epoch

for epoch in range(start_epoch, num_epochs):
    print(f"\n{'='*50}\nEpoch {epoch+1}/{num_epochs} | LR: {optimizer.param_groups[0]['lr']:.6f}")

    # Train
    train_loss, train_acc = train_epoch(
        model, train_dl, criterion, mixup_criterion, mixup_fn,
        optimizer, epoch, num_epochs, warmup_epochs, base_lr, accum_steps,
        ema_model=ema_model, device=device
    )
    training_history['train_loss'].append(train_loss)
    training_history['train_acc'].append(train_acc)

    # Validate (EMA if exists)
    print("\nValidation...")
    val_loss, val_acc = validate(model, val_dl, criterion, ema_model, device=device)
    training_history['val_loss'].append(val_loss)
    training_history['val_acc'].append(val_acc)

    is_best = val_acc > best_val_acc
    if is_best:
        best_val_acc = val_acc

    scheduler.step()

    print(f"\nEpoch {epoch+1} Summary:")
    print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"  Val   Loss: {val_loss:.4f}, Val   Acc: {val_acc:.2f}% (EMA)")
    print(f"  Best Val Acc: {best_val_acc:.2f}%")

    checkpoint = {
        'model_config': model.config,
        'training_config': TRAINING_CONFIG,
        'model_state_dict': model.state_dict(),
        'ema_state_dict': ema_model.state_dict() if ema_model else None,
        'backbone_state_dict': model.backbone.state_dict(),
        'ema_backbone_state_dict': ema_model.model.backbone.state_dict() if ema_model else None,
        'neck_state_dict': model.neck.state_dict() if model.neck else None,
        'ema_neck_state_dict': ema_model.model.neck.state_dict() if (ema_model and model.neck) else None,
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'epoch': epoch + 1,
        'train_loss': train_loss, 'train_acc': train_acc,
        'val_loss': val_loss, 'val_acc': val_acc,
        'best_val_acc': best_val_acc,
        'history': training_history
    }

    # periodic saves
    if (epoch + 1) % 5 == 0 or (epoch + 1) == num_epochs:
        save_checkpoint(checkpoint, os.path.join(checkpoint_dir, 'latest.pth'))
        print("  → latest.pth saved")

    if is_best:
        save_checkpoint(checkpoint, os.path.join(checkpoint_dir, 'best_model.pth'))
        save_pretrained_weights_for_detection(checkpoint_dir, checkpoint, model.config, suffix=" (best)")
        print(f"  → BEST updated! ({val_acc:.2f}%)")

    if (epoch + 1) % 50 == 0:
        save_checkpoint(checkpoint, os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch+1}.pth'))

# done
print("\n" + "="*70); print("TRAINING COMPLETED!"); print("="*70)
print(f"Best Val Acc: {best_val_acc:.2f}%")

print("\n💾 Final ...")
save_checkpoint(checkpoint, os.path.join(checkpoint_dir, 'latest.pth'))
try:
    best_ckpt = torch.load(os.path.join(checkpoint_dir, 'best_model.pth'), map_location='cpu')
    save_pretrained_weights_for_detection(checkpoint_dir, best_ckpt, model.config, suffix=" (FINAL)")
except Exception:
    save_pretrained_weights_for_detection(checkpoint_dir, checkpoint, model.config, suffix=" (FINAL)")

print(f"\n📁 All files: {checkpoint_dir}")
print("\n📄 Saved items:")
print("  - config.json")
print("  - latest.pth")
print("  - best_model.pth")
print("  - pretrained_backbone.pth (for detection )")

